In [ ]:
# -*- coding: utf-8 -*-
# @FileName: openmanus_v7_tech_comments.py
# @Version: Refactored - Async, Decorator Tools, Technical Comments
# @Author: Your Most Loyal & Dedicated Programmer (Refactored)
# @Date: [Current Date]
# @License: Apache 2.0 (Anticipated)
# @Description:
# ==============================================================================================
#  Manus 系统 技术实现说明
# ==============================================================================================
#
# 尊敬的领导，这是我们为电路设计任务精心打造的异步 Agent 核心实现。
# 其核心逻辑遵循了业界标准的 Agentic 循环：感知 -> 规划 -> 行动 -> 观察 -> 响应生成。
# 这种设计模式确保了 Agent 行为的逻辑性和可扩展性。
#
# 系统的核心组件设计如下：
# 1.  `MemoryManager`: 负责管理 Agent 的记忆。这包括用于 LLM 上下文的短期对话历史
#     (我们设计了基于数量的修剪机制以防止上下文过长，确保高效交互)、
#     长期知识片段 (当前为简单 FIFO 队列，为未来引入 RAG 等高级检索策略预留了接口)，
#     以及结构化的电路状态知识 (精细管理元件、连接及 ID 计数器，保证电路状态的准确性)。
# 2.  `LLMInterface`: 封装了与大语言模型 (当前配置为智谱 AI) 的异步交互。我们特别采用了
#     `asyncio.to_thread` 将同步 SDK 调用移至工作线程执行，从而避免阻塞事件循环，
#     保证了系统的响应性能。
# 3.  `OutputParser`: 精心设计用于解析 LLM 返回的文本。其关键职责是在规划阶段准确提取
#     `<think>` 思考过程，并严格校验遵循我们自定义 Schema 的 JSON 计划。在响应生成阶段，
#     它负责提取 `<think>` 和最终的用户回复。我们对 JSON 提取逻辑进行了鲁棒性设计，
#     能有效处理 LLM 可能输出的非标准格式。
# 4.  `ToolExecutor`: 负责按 LLM 规划的顺序，异步协调执行内部工具 (Action 方法)。
#     关键的设计亮点在于“失败中止”策略：若任一工具执行失败 (其 Action 方法返回
#     `status != 'success'`)，将立即停止后续工具的执行，保证了操作的原子性和安全性。
# 5.  内部工具 (Action Methods): 这些是 Agent 执行具体任务的方法 (例如添加元件、建立连接等)。
#     我们采用 `@register_tool` 装饰器模式，不仅标记了这些方法，还附加了它们的描述和
#     参数 Schema。Agent 在初始化时会自动扫描并注册这些工具，极大地提高了可维护性和扩展性。
#
# 主要技术特性概述：
# -   **全面异步化 (`asyncio`)**: 核心流程、LLM 调用和工具执行协调均采用异步设计，以最大化系统吞吐量和响应速度。
# -   **自定义 JSON 规划**: 我们没有直接依赖 LLM 的内置 Function Calling，而是要求 LLM 生成特定格式的 JSON
#     来描述执行计划。这种方式提供了对规划过程更精细的控制，确保了执行的准确性。
# -   **规划重试**: 针对 LLM 调用可能出现的瞬时失败或返回格式错误，我们在规划阶段引入了自动重试机制 (次数可配置)，
#     增强了系统的容错能力。
# -   **工具失败处理**: ToolExecutor 的失败中止机制确保了在复杂任务链中，一旦某一步骤失败，不会继续执行后续无效或危险的操作。
# -   **记忆修剪**: MemoryManager 自动管理短期记忆大小，防止上下文窗口无限增长，保证了与 LLM 交互的效率和成本控制。
# -   **动态工具注册**: 装饰器模式使得添加新工具或修改现有工具变得异常简单和清晰，无需修改核心注册逻辑。
#
# ==============================================================================================


# --- 基础库导入 ---
import re
import os
import json
import time
import logging
import sys
import asyncio
import traceback
import inspect # 用于在运行时发现通过装饰器注册的工具方法
import functools # 用于在装饰器中保留原始函数的重要元信息 (如名称、文档字符串)
from typing import List, Dict, Any, Optional, Tuple, Set, Union
from zhipuai import ZhipuAI # 引入与智谱 AI API 交互的官方 SDK

# --- 全局异步事件循环 ---
# 确保在不同执行环境（如标准脚本、Jupyter Notebook）中都能正确获取或创建事件循环，
# 这是保证异步代码正常运行的基础。
try:
    loop = asyncio.get_running_loop()
except RuntimeError:
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)

# --- 日志系统配置 ---
# 配置了全局日志系统，提供详细的运行时信息，便于调试和监控。
logging.basicConfig(
    level=logging.DEBUG, # 开发阶段设置为 DEBUG 级别以获取最详细信息，生产环境可调整为 INFO 或更高。
    format='%(asctime)s - %(name)s - %(levelname)s [%(module)s.%(funcName)s:%(lineno)d] - %(message)s', # 定义了清晰的日志格式。
    stream=sys.stderr # 将日志输出到 stderr，以避免与程序正常的 stdout 输出混淆。
)
logger = logging.getLogger(__name__)
# 调低了依赖库 (如 zhipuai, httpx) 的日志级别，以减少不必要的噪音，聚焦于我们应用本身的日志。
logging.getLogger("zhipuai").setLevel(logging.WARNING)
logging.getLogger("httpx").setLevel(logging.WARNING)

# --- 异步友好的打印函数 ---
# 设计了一个对异步环境安全的打印函数。
async def async_print(message: str, end: str = '\n', flush: bool = True):
    """在异步环境中安全地向标准输出打印消息，避免潜在的输出交错问题。"""
    # 对于简单的命令行应用，直接写入 sys.stdout 通常是可接受的。
    # 在需要更高并发或与 GUI/Web 框架集成的复杂场景中，可能需要考虑更健壮的日志记录或消息队列机制。
    sys.stdout.write(message + end)
    if flush:
        sys.stdout.flush()

# --- 电路元件数据类 ---
class CircuitComponent:
    """定义电路元件的标准数据结构，封装了元件的核心属性并提供了基础的输入验证。
    使用这个类可以确保元件数据的一致性和有效性。
    """
    __slots__ = ['id', 'type', 'value'] # 使用 __slots__ 可以减少对象实例的内存占用，对可能包含大量元件的场景有性能优势。
    def __init__(self, component_id: str, component_type: str, value: Optional[str] = None):
        # 在初始化时执行严格的输入验证，确保 ID 和类型是有效的非空字符串。
        if not isinstance(component_id, str) or not component_id.strip():
            raise ValueError("元件 ID 必须是有效的非空字符串")
        if not isinstance(component_type, str) or not component_type.strip():
            raise ValueError("元件类型必须是有效的非空字符串")
        # 对 ID 进行规范化处理：去除首尾空格并转为大写，以保证 ID 的唯一性和一致性。
        self.id: str = component_id.strip().upper()
        # 对类型进行规范化处理：去除首尾空格。
        self.type: str = component_type.strip()
        # 对值进行处理：转换为字符串，去除空格。如果原始值为 None 或处理后为空字符串，则存储为 None。
        self.value: Optional[str] = str(value).strip() if value is not None and str(value).strip() else None
        logger.debug(f"成功创建元件对象: {self}")
    def __str__(self) -> str:
        # 定义了对象的字符串表示形式，用于生成用户友好的描述和日志记录。
        value_str = f" (值: {self.value})" if self.value else ""
        return f"元件: {self.type} (ID: {self.id}){value_str}"
    def __repr__(self) -> str:
        # 定义了对象的开发者友好表示形式，主要用于调试。
        return f"CircuitComponent(id='{self.id}', type='{self.type}', value={repr(self.value)})"

# --- 工具注册装饰器 ---
def register_tool(description: str, parameters: Dict[str, Any]):
    """
    这是一个精心设计的装饰器，用于将 Agent 类中的特定方法标记为可供 LLM 调用的工具。
    它接收该工具的自然语言描述和结构化的参数 Schema (遵循类似 OpenAI Function Calling 的格式)。
    这些信息会被附加到被装饰的方法上，使得 Agent 在初始化时能够自动发现并注册这些工具。
    这种方式极大地简化了工具的定义和管理过程。
    """
    def decorator(func):
        # 将描述和参数 Schema 存储在函数对象的自定义属性 `_tool_schema` 中。
        func._tool_schema = {"description": description, "parameters": parameters}
        # 添加一个 `_is_tool` 标记属性，便于后续通过反射机制识别出哪些方法是工具。
        func._is_tool = True
        # 使用 `functools.wraps` 保留原始函数的元信息 (如函数名、文档字符串等)，
        # 这对于调试、文档生成以及维持代码的可读性至关重要。
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            # 这个包装器本身不修改原始函数的行为，其主要作用是在函数对象上附加元数据。
            return func(*args, **kwargs)
        return wrapper
    return decorator


# --- 模块化组件：MemoryManager ---
class MemoryManager:
    """
    记忆管理器，是 Agent 状态信息的核心存储与管理单元。
    它负责维护 Agent 的所有记忆类型：
    - 短期记忆: 存储最近的对话历史，作为 LLM 理解上下文的基础。
    - 长期记忆: 存储从交互中学习到的知识片段或经验总结 (当前为基础实现)。
    - 电路知识库: 结构化地存储当前的电路状态，包括元件、连接和用于生成 ID 的计数器。
    """
    def __init__(self, max_short_term_items: int = 20, max_long_term_items: int = 50):
        logger.info("[MemoryManager] 初始化记忆模块...")
        # 短期记忆容量必须大于 1，以至少容纳一次基本的 System-User-Assistant 交互。
        if max_short_term_items <= 1:
            raise ValueError("max_short_term_items 必须大于 1")
        self.max_short_term_items = max_short_term_items
        self.max_long_term_items = max_long_term_items
        # 短期记忆存储为消息对象的列表。
        self.short_term: List[Dict[str, Any]] = []
        # 长期记忆存储为知识片段字符串的列表 (当前实现为简单的 FIFO 队列)。
        self.long_term: List[str] = []
        # 电路知识库使用字典进行结构化存储。
        self.circuit_knowledge: Dict[str, Any] = {
            "components": {}, # 存储元件对象，键为元件 ID。
            "connections": set(), # 使用集合存储连接，每个连接表示为排序后的元件 ID 对元组，自动处理重复和顺序问题。
            "_component_counters": { # 维护常见元件类型的 ID 生成计数器。
                'R': 0, 'L': 0, 'B': 0, 'S': 0, 'C': 0, 'V': 0, 'G': 0, 'U': 0, 'O': 0,
                'I': 0, 'A': 0, 'D': 0, 'P': 0, 'F': 0, 'H': 0 # 预置了多种常见类型。
            }
        }
        logger.info(f"[MemoryManager] 记忆模块初始化完成。短期上限: {max_short_term_items} 条, 长期上限: {max_long_term_items} 条")

    def add_to_short_term(self, message: Dict[str, Any]):
        """
        向短期记忆中添加一条新消息，并在必要时执行修剪以维持容量限制。
        当前的修剪策略是基于消息数量的：如果超出限制，会移除最旧的非系统 (`role != 'system'`) 消息。
        系统消息通常包含重要的初始指令，需要保留。
        这是一个基础策略，未来可以考虑实现更精细的、基于 Token 数量的修剪策略，以更精确地控制上下文长度。
        """
        logger.debug(f"[MemoryManager] 添加消息到短期记忆 (Role: {message.get('role', 'N/A')}). 当前数量: {len(self.short_term)}")
        self.short_term.append(message)

        # 检查是否超出容量限制。
        current_size = len(self.short_term)
        if current_size > self.max_short_term_items:
            logger.debug(f"[MemoryManager] 短期记忆超限 ({current_size}/{self.max_short_term_items})，执行修剪...")
            items_to_remove = current_size - self.max_short_term_items
            removed_count = 0
            indices_to_remove = []

            # 查找需要移除的最旧的非系统消息的索引。
            # 优先移除对话历史中的用户和助手消息。
            for i in range(len(self.short_term)):
                # 系统消息通常位于列表开头，我们跳过它。
                if self.short_term[i].get("role") != "system":
                    indices_to_remove.append(i)
                if len(indices_to_remove) == items_to_remove:
                    break # 找到了足够数量的消息进行移除。

            # 为了高效且安全地移除元素（避免在迭代时修改列表），我们创建一个新的列表。
            new_short_term = []
            indices_to_remove_set = set(indices_to_remove) # 使用集合进行快速查找。
            removed_roles = []
            for i, item in enumerate(self.short_term):
                if i not in indices_to_remove_set:
                    new_short_term.append(item) # 保留不在移除列表中的消息。
                else:
                    removed_roles.append(item.get('role', 'N/A')) # 记录被移除消息的角色，用于日志。
                    removed_count += 1

            self.short_term = new_short_term # 更新短期记忆列表。
            if removed_count > 0:
                 logger.info(f"[MemoryManager] 短期记忆修剪完成，移除了 {removed_count} 条最旧的非系统消息 (Roles: {removed_roles})。")
            else:
                 # 这种情况可能在 max_short_term_items 设置过低时发生。
                 logger.warning("[MemoryManager] 短期记忆超限但未能找到足够的非系统消息进行移除。请检查 max_short_term_items 设置。")

        logger.debug(f"[MemoryManager] 添加后短期记忆数量: {len(self.short_term)}")

    def add_to_long_term(self, knowledge_snippet: str):
        """向长期记忆中添加知识片段。当前实现采用了简单的先进先出 (FIFO) 策略进行修剪。"""
        # 记录添加操作和当前状态，截断长片段以便于日志阅读。
        logger.debug(f"[MemoryManager] 添加知识到长期记忆: '{knowledge_snippet[:100]}{'...' if len(knowledge_snippet) > 100 else ''}'. 当前数量: {len(self.long_term)}")
        self.long_term.append(knowledge_snippet)
        # 如果超过容量限制，移除列表开头的最旧条目。
        if len(self.long_term) > self.max_long_term_items:
            removed = self.long_term.pop(0)
            logger.info(f"[MemoryManager] 长期记忆超限 ({self.max_long_term_items}), 移除最旧知识: '{removed[:50]}...'")
        logger.debug(f"[MemoryManager] 添加后长期记忆数量: {len(self.long_term)}")

    def get_circuit_state_description(self) -> str:
        """根据电路知识库中的当前数据，生成一份结构清晰、易于阅读的电路状态文本描述。"""
        logger.debug("[MemoryManager] 正在生成电路状态描述...")
        components = self.circuit_knowledge["components"]
        connections = self.circuit_knowledge["connections"]
        num_components = len(components)
        num_connections = len(connections)

        # 特殊处理电路为空的情况。
        if num_components == 0 and num_connections == 0:
            return "【当前电路状态】: 电路为空。"

        # 构建描述文本的各个部分。
        desc_lines = ["【当前电路状态】:"]
        desc_lines.append(f"  - 元件 ({num_components}):")
        if components:
            # 对元件按 ID 进行排序，确保每次生成的描述顺序一致，提高可预测性。
            sorted_ids = sorted(components.keys())
            for cid in sorted_ids: desc_lines.append(f"    - {str(components[cid])}")
        else: desc_lines.append("    (无)")

        desc_lines.append(f"  - 连接 ({num_connections}):")
        if connections:
            # 将连接集合转换为列表并排序，同样是为了保证输出顺序的一致性。
            sorted_connections = sorted(list(connections))
            for c1, c2 in sorted_connections: desc_lines.append(f"    - {c1} <--> {c2}")
        else: desc_lines.append("    (无)")

        description = "\n".join(desc_lines)
        logger.debug("[MemoryManager] 电路状态描述生成完毕。")
        return description

    def get_memory_context_for_prompt(self, recent_long_term_count: int = 5) -> str:
        """
        格式化用于注入 LLM Prompt 的记忆上下文，这部分不包括对话历史
        (对话历史由 Orchestrator 直接处理)。
        它主要包含当前的电路状态描述和部分长期记忆。
        目前的实现很简单，仅使用了最近的 N 条长期记忆。这是一个基础策略，
        理想情况下，未来的版本应采用基于当前查询相关性的 RAG (Retrieval-Augmented Generation)
        技术来动态检索最相关的长期记忆片段，以提供更精确的上下文。
        """
        logger.debug("[MemoryManager] 正在格式化记忆上下文用于 Prompt...")
        # 获取当前的电路状态文本描述。
        circuit_desc = self.get_circuit_state_description()

        # --- 长期记忆处理 (基础版：仅使用最近 N 条) ---
        long_term_str = ""
        if self.long_term:
            # 计算实际要提取的数量，不超过设定的上限和实际存储的数量。
            actual_count = min(recent_long_term_count, len(self.long_term))
            if actual_count > 0:
                # 从列表末尾提取最近的 N 条记录。
                recent_items = self.long_term[-actual_count:]
                long_term_str = "\n\n【近期经验总结 (仅显示最近 N 条)】\n" + "\n".join(f"- {item}" for item in recent_items)
                logger.debug(f"[MemoryManager] 已提取最近 {len(recent_items)} 条长期记忆 (基础模式)。")
            # else: 不需要提取 (数量为 0 或负数)
        # else: 长期记忆为空

        # 在上下文中明确告知 LLM 当前记忆检索方式的局限性。
        long_term_str += "\n(注: 当前仅使用最近期记忆，未来版本将实现基于相关性的检索)"

        # 组合电路描述和格式化后的长期记忆上下文。
        context = f"{circuit_desc}{long_term_str}".strip()
        logger.debug(f"[MemoryManager] 记忆上下文 (电路+长期) 格式化完成。")
        return context

    def generate_component_id(self, component_type: str) -> str:
        """
        为给定类型的元件生成一个在当前电路中唯一的 ID。
        此方法维护一个从元件类型（包括常见的中英文别名）到 ID 前缀的映射，
        并为每个前缀维护一个递增的计数器，从而生成如 "R1", "R2", "C1", "V1" 等形式的 ID。
        设计中包含了对输入类型的清理和最长匹配逻辑，以提高对不同用户输入的适应性和鲁棒性。
        同时，它还会检查生成的 ID 是否与用户手动添加的 ID 冲突。
        """
        logger.debug(f"[MemoryManager] 正在为类型 '{component_type}' 生成唯一 ID...")
        # 定义类型关键字到 ID 前缀的映射，覆盖了多种常见表达方式。
        type_map = {
            "resistor": "R", "电阻": "R", "capacitor": "C", "电容": "C",
            "battery": "B", "电池": "B", "voltage source": "V", "voltage": "V",
            "电压源": "V", "电压": "V", "led": "L", "发光二极管": "L", "switch": "S",
            "开关": "S", "ground": "G", "地": "G", "ic": "U", "chip": "U", "芯片": "U",
            "集成电路": "U", "inductor": "I", "电感": "I", "current source": "A",
            "电流源": "A", "diode": "D", "二极管": "D", "potentiometer": "P", "电位器": "P",
            "fuse": "F", "保险丝": "F", "header": "H", "排针": "H",
            "component": "O", "元件": "O", # 为未知或通用类型设置了默认前缀 'O'。
        }
        # 确保映射中定义的所有前缀都在计数器字典中有初始值 (通常为 0)。
        for code in type_map.values():
            if code not in self.circuit_knowledge["_component_counters"]:
                 self.circuit_knowledge["_component_counters"][code] = 0

        # 清理输入类型字符串：去除首尾空格并转为小写，便于匹配。
        cleaned_type = component_type.strip().lower()
        type_code = "O" # 默认使用通用前缀。
        best_match_len = 0
        # 采用最长匹配原则来确定类型代码，避免短关键字覆盖长关键字的情况
        # (例如，确保 "voltage source" 匹配到 "V" 而不是 "S")。
        for keyword, code in type_map.items():
            if keyword in cleaned_type and len(keyword) > best_match_len:
                type_code = code
                best_match_len = len(keyword)

        # 如果最终匹配到的是通用前缀 'O'，且输入类型并非明确的通用类型词，则记录警告。
        if type_code == "O" and cleaned_type not in ["component", "元件"]:
             logger.warning(f"[MemoryManager] 未找到类型 '{component_type}' 的特定前缀，将使用通用前缀 'O'。")

        MAX_ID_ATTEMPTS = 100 # 设置一个尝试上限，防止在极端情况下 (例如 ID 空间被用户手动 ID 占满) 陷入无限循环。
        for attempt in range(MAX_ID_ATTEMPTS):
            # 递增对应类型前缀的计数器。
            self.circuit_knowledge["_component_counters"][type_code] += 1
            # 基于前缀和当前计数生成候选 ID。
            gen_id = f"{type_code}{self.circuit_knowledge['_component_counters'][type_code]}"
            # 检查生成的 ID 是否已经在元件字典中存在 (可能是用户手动添加的)。
            if gen_id not in self.circuit_knowledge["components"]:
                logger.debug(f"[MemoryManager] 生成唯一 ID: '{gen_id}' (尝试 {attempt + 1})")
                return gen_id # 找到了一个未被使用的 ID，成功返回。
            # 如果 ID 已存在，记录警告并继续尝试下一个。
            logger.warning(f"[MemoryManager] ID '{gen_id}' 已存在（可能是手动添加的），尝试下一个。")

        # 如果在达到尝试上限后仍未找到可用的 ID，则抛出运行时错误，指示存在问题。
        raise RuntimeError(f"未能为类型 '{component_type}' (代码 '{type_code}') 生成唯一 ID ({MAX_ID_ATTEMPTS} 次尝试后)。电路中可能存在大量冲突的 ID。")

    def clear_circuit(self):
        """彻底清空当前电路的所有状态信息，包括所有元件和连接，并将所有元件类型的 ID 计数器重置为 0。
        这是一个用于重置 Agent 电路设计状态的操作。"""
        logger.info("[MemoryManager] 正在清空电路状态...")
        # (可选) 记录清空前的状态统计信息。
        comp_count = len(self.circuit_knowledge["components"])
        conn_count = len(self.circuit_knowledge["connections"])
        # 重置元件字典和连接集合。
        self.circuit_knowledge["components"] = {}
        self.circuit_knowledge["connections"] = set()
        # 将所有类型的 ID 计数器重置为 0。
        self.circuit_knowledge["_component_counters"] = {k: 0 for k in self.circuit_knowledge["_component_counters"]}
        logger.info(f"[MemoryManager] 电路状态已清空 (移除了 {comp_count} 个元件, {conn_count} 个连接，并重置了所有 ID 计数器)。")

# --- 模块化组件：LLMInterface ---
class LLMInterface:
    """
    封装与大语言模型 (LLM) API 的异步交互逻辑。
    这个类的目的是提供一个清晰、独立的接口来处理所有与 LLM 通信相关的细节，
    包括 API 认证、请求参数构建、异步调用执行以及初步的响应处理。
    当前实现针对智谱 AI 的 SDK。
    """
    def __init__(self, api_key: str, model_name: str = "glm-4-flash", default_temperature: float = 0.1, default_max_tokens: int = 4095):
        logger.info(f"[LLMInterface] 初始化 LLM 接口，目标模型: {model_name}")
        if not api_key: raise ValueError("智谱 AI API Key 不能为空")
        try:
            # 实例化智谱 AI 的客户端对象，用于后续的 API 调用。
            self.client = ZhipuAI(api_key=api_key)
            # TODO: 未来可以调研 ZhipuAI SDK 是否提供原生异步客户端，如果可用且性能更优，应优先考虑切换。
            logger.info("[LLMInterface] 智谱 AI 客户端初始化成功。")
        except Exception as e:
            # 客户端初始化失败是一个严重问题，需要记录详细错误并向上抛出，以便系统能感知到此问题。
            logger.critical(f"[LLMInterface] 初始化智谱 AI 客户端失败: {e}", exc_info=True)
            raise ConnectionError(f"初始化智谱 AI 客户端失败: {e}") from e
        # 存储模型名称和默认的调用参数，方便管理和复用。
        self.model_name = model_name
        self.default_temperature = default_temperature
        self.default_max_tokens = default_max_tokens
        logger.info(f"[LLMInterface] LLM 接口初始化完成 (Model: {model_name}, Temp: {default_temperature}, MaxTokens: {default_max_tokens})。")

    async def call_llm(self, messages: List[Dict[str, Any]], use_tools: bool = False, tool_choice: Optional[str] = None) -> Any:
        """
        执行对 LLM API 的异步调用。
        在本 Agent 的设计中，规划阶段 (`use_tools=False`) 依赖于解析 LLM 输出的自定义 JSON，
        而非 SDK 内置的 `tools` 功能。因此，`use_tools=False` 是常用路径。
        保留 `use_tools=True` 的分支是为了兼容未来可能需要利用 SDK 管理工具调用的场景。
        """
        # 准备传递给 SDK 的调用参数字典。
        call_args = {
            "model": self.model_name,
            "messages": messages,
            "temperature": self.default_temperature,
            "max_tokens": self.default_max_tokens,
        }
        if use_tools:
            # 此分支在当前的规划流程中并未激活。
            logger.warning("[LLMInterface] 注意: 请求使用 SDK tools 参数。这需要确保 tools 定义已正确传递。")
            # 示例性代码，实际使用时需要传递正确的工具定义:
            # if passed_tools_definition: call_args["tools"] = passed_tools_definition
            # if tool_choice: call_args["tool_choice"] = tool_choice
            pass
        else:
            # 这是我们自定义 JSON 规划模式下的日志。
            logger.info(f"[LLMInterface] 准备异步调用 LLM ({self.model_name}，自定义 JSON/无内置工具模式)...")

        logger.debug(f"[LLMInterface] 发送的消息条数: {len(messages)}")
        # 仅在需要深度调试 LLM 输入时取消此行注释。
        # logger.debug(f"[LLMInterface] 消息列表: {messages}")

        try:
            start_time = time.monotonic() # 使用 monotonic 时钟精确测量调用耗时。
            # 为了在异步环境中使用同步的 SDK 方法而不阻塞事件循环，我们采用了 `asyncio.to_thread`。
            # 这个函数会将同步方法 `self.client.chat.completions.create` 放到一个独立的线程中执行。
            response = await asyncio.to_thread(
                self.client.chat.completions.create, # 这是同步的 SDK 方法。
                **call_args
            )
            # 如果 ZhipuAI SDK 未来提供了原生的异步方法 (例如 `create_async`)，
            # 切换到原生异步方法通常是更优的选择:
            # response = await self.client.chat.completions.create_async(**call_args)

            duration = time.monotonic() - start_time
            logger.info(f"[LLMInterface] LLM 异步调用成功。耗时: {duration:.3f} 秒。")

            # 对返回的响应进行基本的有效性检查和信息记录。
            if response:
                if response.usage: # 记录 token 使用情况，有助于成本控制和性能分析。
                    logger.info(f"[LLMInterface] Token 统计: Prompt={response.usage.prompt_tokens}, Completion={response.usage.completion_tokens}, Total={response.usage.total_tokens}")
                if response.choices: # 记录完成原因，有助于判断 LLM 是否正常结束或被截断。
                    logger.info(f"[LLMInterface] 完成原因: {response.choices[0].finish_reason}")
                else:
                     logger.warning("[LLMInterface] LLM 响应中缺少 'choices' 字段，可能表示请求失败或响应格式异常。")
            else:
                 # LLM API 返回 None 是一个严重的错误信号。
                 logger.error("[LLMInterface] LLM API 调用返回了 None！")
                 raise ConnectionError("LLM API call returned None.")

            return response
        except Exception as e:
            # 捕获并记录在 LLM 调用过程中发生的任何异常。
            logger.error(f"[LLMInterface] LLM API 异步调用失败: {e}", exc_info=True)
            # 将异常向上抛出，由调用者 (Orchestrator) 负责处理，例如执行重试逻辑。
            raise

# --- 模块化组件：OutputParser ---
class OutputParser:
    """
    负责解析从 LLM 返回的响应内容。
    核心任务是从原始的、可能混杂的文本输出中，精确地提取出结构化的信息。
    对于规划阶段，需要提取 `<think>` 块中的思考过程和符合我们自定义 Schema 的 JSON 计划。
    对于最终响应阶段，则需要提取 `<think>` 块和最终的用户回复文本。
    我们特别注重 JSON 提取的鲁棒性。
    """
    def __init__(self):
        logger.info("[OutputParser] 初始化输出解析器 (用于自定义 JSON 和文本解析)。")

    def parse_planning_response(self, response_message: Any) -> Tuple[str, Optional[Dict[str, Any]], str]:
        """
        解析规划阶段 (第一次 LLM 调用) 的响应。
        此方法预期 LLM 的输出严格遵循 `<think>...</think> JSON_OBJECT` 的格式。
        它不仅要提取出 `<think>` 内容和 JSON 部分，还要对 JSON 对象进行严格的结构和类型验证。
        为了应对 LLM 可能在 JSON 前后添加无关文本的情况，我们实现了一种鲁棒的 JSON 查找逻辑：
        定位第一个 `{` 或 `[`，然后通过匹配括号层级找到对应的结束符 `}` 或 `]`。

        Args:
            response_message: LLM 返回的消息对象 (通常是 Pydantic 模型或类似结构)。

        Returns:
            一个元组，包含:
            - (str) 提取出的思考过程文本。
            - (Optional[Dict[str, Any]]) 解析并验证成功的 JSON 计划 (如果失败则为 None)。
            - (str) 如果发生错误，则包含错误信息；否则为空字符串。
        """
        logger.debug("[OutputParser] 开始解析规划响应 (自定义 JSON 模式)...")
        thinking_process = "未能提取思考过程。" # 设置默认值以防提取失败。
        plan = None
        error_message = ""

        # 首先进行基本的输入有效性检查。
        if response_message is None:
            error_message = "LLM 响应对象为 None。"
            logger.error(f"[OutputParser] 解析失败: {error_message}")
            return thinking_process, None, error_message

        # 从响应对象中获取核心的文本内容。
        # 假设使用的是包含 `content` 属性的对象 (如 Pydantic 模型)。
        raw_content = getattr(response_message, 'content', None)

        # 检查内容是否为空或仅包含空白。
        if not raw_content or not raw_content.strip():
            # 如果内容为空，检查是否意外地包含了 `tool_calls` (这不符合自定义 JSON 模式)。
            tool_calls = getattr(response_message, 'tool_calls', None)
            if tool_calls:
                 error_message = "LLM 响应内容为空，但意外地包含了 tool_calls。"
            else:
                 error_message = "LLM 响应内容为空或仅包含空白字符。"
            logger.error(f"[OutputParser] 解析失败: {error_message}")
            return thinking_process, None, error_message

        # --- 提取 <think> 块 ---
        # 使用正则表达式查找并提取 `<think>...</think>` 标签之间的内容。
        think_match = re.search(r'<think>(.*?)</think>', raw_content, re.IGNORECASE | re.DOTALL)
        json_part_start_index = 0 # 默认假设 JSON 从响应开头开始。
        if think_match:
            thinking_process = think_match.group(1).strip()
            json_part_start_index = think_match.end() # 如果找到 <think>，JSON 应紧随其后。
            logger.debug("[OutputParser] 成功提取 <think> 内容。")
        else:
            # 如果未找到 <think> 标签，记录警告，并尝试从头开始解析 JSON。
            thinking_process = "警告：未找到 <think> 标签，将尝试解析后续内容为 JSON。"
            logger.warning(f"[OutputParser] {thinking_process}")
            json_part_start_index = 0

        # --- 提取并解析 JSON 部分 ---
        # 获取 <think> 标签之后（或整个响应，如果没有 <think>）的潜在 JSON 内容。
        potential_json_part = raw_content[json_part_start_index:].strip()
        # 记录待解析内容的前 1000 字符，这对于调试 LLM 输出的格式问题非常有帮助。
        logger.debug(f"[OutputParser] 提取出的待解析 JSON 字符串 (前 1000 字符): >>>\n{potential_json_part[:1000]}{'...' if len(potential_json_part) > 1000 else ''}\n<<<")

        if not potential_json_part:
            error_message = "提取出的潜在 JSON 内容为空。"
            logger.error(f"[OutputParser] 解析失败: {error_message}")
            return thinking_process, None, error_message

        final_json_string = "" # 用于存储最终精确提取的 JSON 字符串。
        try:
            # 进行预处理：尝试去除常见的 Markdown 代码块标记 (```json ... ``` 或 ``` ... ```)。
            json_string_to_parse = potential_json_part
            if json_string_to_parse.startswith("```json"):
                json_string_to_parse = json_string_to_parse[len("```json"):].strip()
            if json_string_to_parse.startswith("```"):
                json_string_to_parse = json_string_to_parse[len("```"):].strip()
            if json_string_to_parse.endswith("```"):
                json_string_to_parse = json_string_to_parse[:-len("```")].strip()

            # 实现鲁棒的 JSON 查找逻辑：
            # 目标是找到第一个有效的 JSON 对象或数组的开始 (`{` 或 `[`)，
            # 并通过跟踪括号的嵌套层级和字符串引号状态，精确地找到与之匹配的结束符 (`}` 或 `]`)。
            # 这种方法可以有效处理 JSON 前后夹杂非 JSON 文本的情况。
            json_start = -1
            json_end = -1
            brace_level = 0     # 大括号嵌套层级计数器。
            square_level = 0    # 方括号嵌套层级计数器。
            in_string = False   # 标记当前是否在 JSON 字符串内部。
            string_char = ''    # 记录当前字符串使用的引号类型 (' 或 ")。
            possible_start = -1 # 记录找到的第一个 `{` 或 `[` 的位置。

            # 确定 JSON 的起始位置。
            first_brace = json_string_to_parse.find('{')
            first_square = json_string_to_parse.find('[')
            # 选择最先出现的 `{` 或 `[` 作为起始点。
            if first_brace != -1 and (first_square == -1 or first_brace < first_square):
                possible_start = first_brace
            elif first_square != -1 and (first_brace == -1 or first_square < first_brace):
                 possible_start = first_square

            if possible_start == -1:
                # 如果连起始括号都找不到，说明无法解析。
                raise json.JSONDecodeError("无法在文本中定位 JSON 对象或数组的起始。", json_string_to_parse, 0)

            json_start = possible_start
            start_char = json_string_to_parse[json_start] # 记录起始括号类型。

            # 遍历字符串，从起始位置开始查找匹配的结束括号。
            for i in range(json_start, len(json_string_to_parse)):
                char = json_string_to_parse[i]
                if in_string:
                    # 处理在字符串内部的情况，需要正确处理转义字符 `\`。
                    # 只有当当前字符是匹配的引号，并且前一个字符不是转义符时，才认为字符串结束。
                    if char == string_char and (i == json_start or json_string_to_parse[i-1] != '\\'):
                        in_string = False # 离开字符串状态。
                else:
                    # 如果不在字符串内部...
                    if char == '"' or char == "'": # 进入字符串状态。
                        in_string = True
                        string_char = char
                    # 根据起始括号类型，更新相应的嵌套层级计数器。
                    elif char == '{' and start_char == '{': brace_level += 1
                    elif char == '}' and start_char == '{': brace_level -= 1
                    elif char == '[' and start_char == '[': square_level += 1
                    elif char == ']' and start_char == '[': square_level -= 1

                # 检查是否找到了匹配的结束括号 (层级归零) 并且当前不在字符串内部。
                if not in_string and (start_char == '{' and brace_level == 0 and char == '}') or \
                   (start_char == '[' and square_level == 0 and char == ']'):
                    json_end = i + 1 # 结束位置是当前字符的下一个位置。
                    break # 找到了，退出循环。

            if json_end == -1:
                # 如果遍历结束仍未找到匹配的结束符，说明 JSON 格式不完整或错误。
                raise json.JSONDecodeError("无法在文本中找到匹配的 JSON 结束符。JSON 可能不完整或格式错误。", json_string_to_parse, len(json_string_to_parse)-1)

            # 提取出通过括号匹配精确找到的 JSON 字符串。
            final_json_string = json_string_to_parse[json_start:json_end]
            logger.debug(f"[OutputParser] 精准提取的 JSON 字符串: >>>\n{final_json_string}\n<<<")

            # 使用标准库 `json.loads` 解析提取出的字符串。
            parsed_json = json.loads(final_json_string)
            logger.debug("[OutputParser] JSON 字符串解析成功。")

            # --- 严格验证 JSON 结构 ---
            # 在成功解析 JSON 后，执行严格的结构和类型验证，确保其符合我们定义的 Schema。
            # 这是保证后续处理流程正确性的关键步骤。
            if not isinstance(parsed_json, dict):
                raise ValueError("解析结果不是一个 JSON 对象 (字典)。")
            # 检查必需的布尔字段 `is_tool_calls`。
            if "is_tool_calls" not in parsed_json or not isinstance(parsed_json["is_tool_calls"], bool):
                raise ValueError("JSON 对象缺少必需的布尔字段 'is_tool_calls'。")
            tool_list = parsed_json.get("tool_list")
            # 如果计划调用工具 (`is_tool_calls` 为 true)...
            if parsed_json["is_tool_calls"]:
                # 验证 `tool_list` 是否为非空列表。
                if not isinstance(tool_list, list): raise ValueError("当 'is_tool_calls' 为 true 时, 'tool_list' 字段必须是一个列表。")
                if not tool_list: raise ValueError("当 'is_tool_calls' 为 true 时, 'tool_list' 列表不能为空。")
                indices_set = set() # 用于检查工具调用 `index` 的唯一性。
                # 逐项检查 `tool_list` 中的每个工具调用对象。
                for i, tool_item in enumerate(tool_list):
                    if not isinstance(tool_item, dict): raise ValueError(f"'tool_list' 中索引 {i} 的元素不是字典。")
                    # 验证 `toolname`, `params`, `index` 字段的存在性和类型。
                    if not tool_item.get("toolname") or not isinstance(tool_item["toolname"], str): raise ValueError(f"'tool_list' 中索引 {i} 缺少有效的 'toolname' 字符串。")
                    if "params" not in tool_item or not isinstance(tool_item["params"], dict): raise ValueError(f"'tool_list' 中索引 {i} 缺少 'params' 字典 (如果无参数，应为空对象 {{}})。")
                    if not tool_item.get("index") or not isinstance(tool_item["index"], int) or tool_item["index"] <= 0: raise ValueError(f"'tool_list' 中索引 {i} 缺少有效正整数 'index'。")
                    # 检查 `index` 是否重复。
                    if tool_item['index'] in indices_set: raise ValueError(f"'tool_list' 中索引 {i} 的 'index' 值 {tool_item['index']} 与之前的重复。")
                    indices_set.add(tool_item['index'])
                # 可选：检查 `index` 是否从 1 开始且连续。不符合规范时仅记录警告，不阻止执行。
                max_index = max(indices_set) if indices_set else 0
                if len(indices_set) != max_index or set(range(1, max_index + 1)) != indices_set:
                     logger.warning(f"[OutputParser] 验证警告: 'tool_list' 中的 'index' ({sorted(list(indices_set))}) 不连续或不从 1 开始。Agent 仍会按 index 排序执行。")
            # 如果计划不调用工具 (`is_tool_calls` 为 false)...
            else:
                # 验证 `tool_list` 是否为 null 或空列表。
                if tool_list is not None and not isinstance(tool_list, list): raise ValueError("当 'is_tool_calls' 为 false 时, 'tool_list' 字段必须是 null 或列表。")
                if isinstance(tool_list, list) and tool_list: raise ValueError("当 'is_tool_calls' 为 false 时, 'tool_list' 必须为空列表 [] 或 null。")

            # 检查 `direct_reply` 字段。
            direct_reply = parsed_json.get("direct_reply")
            if not parsed_json["is_tool_calls"]:
                # 如果不调用工具，则 `direct_reply` 必须是一个有效的非空字符串。
                if not isinstance(direct_reply, str) or not direct_reply.strip():
                    raise ValueError("当 'is_tool_calls' 为 false 时, 必须提供有效的非空 'direct_reply' 字符串。")
            else: # 如果调用工具...
                # `direct_reply` 必须是 null 或一个可选的说明性字符串。
                if direct_reply is not None and not isinstance(direct_reply, str): raise ValueError("当 'is_tool_calls' 为 true 时, 'direct_reply' 字段必须是 null 或字符串。")

            # 所有验证都通过了，将解析后的 JSON 对象赋给 `plan`。
            plan = parsed_json
            logger.info("[OutputParser] 自定义 JSON 计划解析和验证成功！")

        except json.JSONDecodeError as json_err:
            # 捕获 JSON 解析过程中的错误。
            error_message = f"解析 JSON 失败: {json_err}。请检查 LLM 输出的 JSON 部分是否符合标准。Raw JSON string (截断): '{potential_json_part[:200]}...'"
            logger.error(f"[OutputParser] JSON 解析失败: {error_message}")
        except ValueError as validation_err:
            # 捕获我们自定义的结构验证错误。
            error_message = f"JSON 结构验证失败: {validation_err}。"
            logger.error(f"[OutputParser] JSON 结构验证失败: {error_message} JSON content (可能不完整): {final_json_string if final_json_string else potential_json_part[:200]}")
        except Exception as e:
            # 捕获在解析或验证过程中发生的任何其他未预料到的错误。
            error_message = f"解析规划响应时发生未知错误: {e}"
            logger.error(f"[OutputParser] 解析时未知错误: {error_message}", exc_info=True)

        return thinking_process, plan, error_message

    def _parse_llm_text_content(self, text_content: str) -> Tuple[str, str]:
        """
        从 LLM 的最终文本响应 (通常是第二次调用生成的报告) 中解析思考过程 (`<think>...</think>`) 和正式回复文本。
        这个方法的逻辑相对简单，因为它处理的是格式更规范的最终输出。
        """
        logger.debug("[OutputParser._parse_llm_text_content] 正在解析最终文本内容...")
        if not text_content: return "思考过程为空。", "回复内容为空。" # 处理空输入。

        thinking_process = "未能提取思考过程。" # 设置默认值。
        formal_reply = text_content.strip() # 默认将整个文本内容视为正式回复。

        # 尝试使用正则表达式查找 `<think>` 标签。
        think_match = re.search(r'<think>(.*?)</think>', text_content, re.IGNORECASE | re.DOTALL)
        if think_match:
            # 如果找到，提取标签内的内容作为思考过程。
            thinking_process = think_match.group(1).strip()
            # 将 `<think>` 标签结束之后的所有内容作为正式回复。
            formal_reply = text_content[think_match.end():].strip()
            # 检查 `<think>` 标签之前是否存在非预期的文本内容。
            content_before_think = text_content[:think_match.start()].strip()
            if content_before_think:
                # 如果存在，记录警告，当前选择忽略这部分内容。
                logger.warning(f"[OutputParser._parse_llm_text_content] 在 <think> 标签之前检测到非空白内容: '{content_before_think[:50]}...'。这部分内容已被忽略。")
        else:
            # 如果未找到 `<think>` 标签，将整个内容视为回复，并记录警告。
            logger.warning("[OutputParser._parse_llm_text_content] 未找到 <think>...</think> 标签。将整个内容视为正式回复。")
            thinking_process = "未能提取思考过程 - LLM 可能未按预期包含<think>标签。"

        # 确保即使提取失败，也返回非空的默认字符串，避免后续处理出错。
        thinking_process = thinking_process if thinking_process else "提取的思考过程为空白。"
        formal_reply = formal_reply if formal_reply else "LLM 未生成最终报告内容。"

        logger.debug(f"[OutputParser._parse_llm_text_content] 解析结果 - 思考长度: {len(thinking_process)}, 回复长度: {len(formal_reply)}")
        return thinking_process, formal_reply

# --- 模块化组件：ToolExecutor ---
class ToolExecutor:
    """
    负责按照规划顺序，可靠地执行 Agent 的内部工具 (Action 方法)。
    它接收一个模拟的工具调用列表 (由 Orchestrator 根据 LLM 的 JSON 计划构建)，
    然后异步地、按顺序地执行这些调用。
    一个核心的设计原则是“失败中止” (Fail Fast / Abort on Failure)：
    如果序列中的任何一个工具执行失败 (其 Action 方法返回 `status != 'success'`)，
    将立即停止执行后续的所有工具，并将已执行的结果返回。
    这种机制对于保证复杂任务链的执行安全性和一致性至关重要。
    """
    # 需要 Agent 实例的引用，以便能够调用在其上定义的 Action 方法。
    def __init__(self, agent_instance: 'CircuitDesignAgent'): # Reference the agent class name
        logger.info("[ToolExecutor] 初始化工具执行器 (支持异步和失败中止)。")
        # 校验传入的是否是正确的 Agent 实例类型。
        if not isinstance(agent_instance, CircuitDesignAgent):
            raise TypeError("ToolExecutor 需要一个 CircuitDesignAgent 实例。")
        self.agent_instance = agent_instance
        # 检查 Agent 实例是否具备必需的 MemoryManager 组件 (虽然 ToolExecutor 不直接使用，但 Action 方法可能需要)。
        if not hasattr(agent_instance, 'memory_manager') or not isinstance(agent_instance.memory_manager, MemoryManager):
            raise TypeError("Agent 实例缺少有效的 MemoryManager。")
        # 可以选择性地存储 MemoryManager 的引用，如果 ToolExecutor 需要直接访问它的话。
        # self.memory_manager = agent_instance.memory_manager

    async def execute_tool_calls(self, mock_tool_calls: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
        """
        按顺序异步协调执行列表中的所有模拟工具调用。
        实现了失败中止逻辑：一旦某个工具执行失败，将不再执行后续工具。

        Args:
            mock_tool_calls: 一个模拟的 ToolCall 对象列表。每个对象应包含 'id' 和 'function'
                             字段，其中 'function' 是一个包含 'name' (工具方法名) 和
                             'arguments' (JSON 字符串格式的参数) 的字典。

        Returns:
            一个列表，其中包含**实际被执行了的**工具调用的结果。
            每个列表项是一个字典，包含 'tool_call_id' (对应输入的模拟 ID) 和 'result' 字典。
            'result' 字典应至少包含 'status' ('success' 或 'failure') 和 'message' 字段。
        """
        logger.info(f"[ToolExecutor] 准备异步执行最多 {len(mock_tool_calls)} 个工具调用 (按顺序，失败中止)...")
        execution_results = [] # 用于存储每个已执行工具的返回结果。

        if not mock_tool_calls:
            logger.info("[ToolExecutor] 没有工具需要执行。")
            return []

        total_tools = len(mock_tool_calls)
        # 按顺序迭代计划中的每个模拟工具调用。
        for i, mock_call in enumerate(mock_tool_calls):
            current_tool_index = i + 1
            function_name = "unknown_function" # 设置默认值
            tool_call_id = mock_call.get('id', f'mock_id_{i}') # 使用提供的 ID 或生成一个备用 ID 用于日志和结果关联。
            action_result = None # 用于存储 Action 方法的返回值。
            arguments = {} # 用于存储解析后的参数字典。
            tool_display_name = "未知工具" # 用于更友好的日志和用户界面输出。

            try:
                # --- 1. 解析模拟 ToolCall 对象结构 ---
                func_info = mock_call.get('function')
                # 验证接收到的 `mock_call` 对象的结构是否符合预期 (包含 function 及其 name 和 arguments)。
                if not isinstance(func_info, dict) or 'name' not in func_info or 'arguments' not in func_info:
                     err_msg = f"模拟 ToolCall 对象结构无效。缺少 'function' 或其 'name'/'arguments'。对象: {mock_call}"
                     logger.error(f"[ToolExecutor] {err_msg}")
                     # 即使在解析阶段失败，也要构造一个符合标准格式的失败结果。
                     action_result = {"status": "failure", "message": "错误: 内部工具调用结构无效。", "error": {"type": "MalformedMockCall", "details": err_msg}}
                     execution_results.append({"tool_call_id": tool_call_id, "result": action_result})
                     await async_print(f"  ❌ [{current_tool_index}/{total_tools}] 内部错误: 工具调用结构无效。")
                     break # 结构错误无法继续，中止后续执行。

                function_name = func_info['name'] # 获取工具名称
                function_args_str = func_info['arguments'] # 获取 JSON 字符串格式的参数
                # 美化工具名称，用于显示。
                tool_display_name = function_name.replace('_tool', '').replace('_', ' ').title()
                logger.info(f"[ToolExecutor] 处理工具调用 {current_tool_index}/{total_tools}: Name='{function_name}', MockID='{tool_call_id}'")
                logger.debug(f"[ToolExecutor] 参数 JSON 字符串: '{function_args_str}'")

                await async_print(f"  [{current_tool_index}/{total_tools}] 准备执行: {tool_display_name}...")

                # --- 2. 解析参数 JSON 字符串 ---
                try:
                    # 如果参数字符串非空，则使用 `json.loads` 解析为 Python 字典；否则视为空参数，即空字典。
                    arguments = json.loads(function_args_str) if function_args_str else {}
                    # 确保解析结果确实是一个字典。
                    if not isinstance(arguments, dict):
                         raise TypeError("参数必须是 JSON 对象 (字典)")
                    logger.debug(f"[ToolExecutor] 参数解析成功: {arguments}")
                except (json.JSONDecodeError, TypeError) as json_err:
                    # 如果参数 JSON 字符串格式错误或解析结果不是字典，则记录错误并标记为失败。
                    err_msg = f"工具 '{function_name}' 的参数 JSON 解析失败: {json_err}. Raw: '{function_args_str}'"
                    logger.error(f"[ToolExecutor] 参数解析错误: {err_msg}", exc_info=True)
                    action_result = {"status": "failure", "message": f"错误: 工具 '{function_name}' 的参数格式错误。", "error": {"type": "ArgumentParsing", "details": err_msg}}
                    await async_print(f"  ❌ [{current_tool_index}/{total_tools}] 操作失败: {tool_display_name}. 错误: 参数解析失败。")
                    execution_results.append({"tool_call_id": tool_call_id, "result": action_result})
                    break # 参数错误，中止后续执行。

                # --- 3. 查找并执行对应的 Action 方法 ---
                # 使用 `getattr` 在 Agent 实例上动态查找名称与 `function_name` 匹配的方法。
                tool_action_method = getattr(self.agent_instance, function_name, None)
                # 检查找到的是否是一个可调用的方法。
                if callable(tool_action_method):
                    # 找到了对应的 Action 方法实现。
                    logger.info(f"[ToolExecutor] >>> 正在调用 Action 方法: '{function_name}' (对应 Mock ID: '{tool_call_id}')")
                    logger.debug(f"[ToolExecutor] 传递给 '{function_name}' 的参数: {arguments}")
                    await async_print(f"  [{current_tool_index}/{total_tools}] ⚙️ 正在执行 '{tool_display_name}'...")
                    try:
                        # 假设 Action 方法本身是同步定义的 (`def` 而非 `async def`)。
                        # 使用 `asyncio.to_thread` 将其调用放入线程池中执行，避免阻塞异步事件循环。
                        # 如果 Action 方法本身是异步 (`async def`)，则应直接使用 `await tool_action_method(arguments=arguments)`。
                        action_result = await asyncio.to_thread(tool_action_method, arguments=arguments)

                        # 严格检查 Action 方法的返回值是否符合预期的字典结构 (必须包含 'status' 和 'message')。
                        if not isinstance(action_result, dict) or 'status' not in action_result or 'message' not in action_result:
                            err_msg = f"Action '{function_name}' 返回的结构无效 (缺少 'status' 或 'message'): {str(action_result)[:200]}... 将强制标记为失败。"
                            logger.error(f"[ToolExecutor] Action 返回结构错误: {err_msg}")
                            # 即使返回结构错误，也构造一个标准格式的失败结果字典。
                            action_result = {"status": "failure", "message": f"错误: 工具 '{function_name}' 返回结果结构无效。", "error": {"type": "InvalidActionResult", "details": err_msg}}
                        else:
                             # Action 返回了有效的结构。
                             logger.info(f"[ToolExecutor] Action '{function_name}' 执行完毕。状态: {action_result.get('status', 'N/A')}")

                        # 根据执行状态 ('success' 或 'failure') 更新用户界面反馈。
                        status_icon = "✅" if action_result.get("status") == "success" else "❌"
                        # 截断消息预览，防止过长的消息刷屏。
                        msg_preview = action_result.get('message', '无消息')[:80] + ('...' if len(action_result.get('message', '')) > 80 else '')
                        await async_print(f"  {status_icon} [{current_tool_index}/{total_tools}] 操作完成: {tool_display_name}. 结果: {msg_preview}")

                    except TypeError as te:
                        # 捕获在调用 Action 方法时，由于传入参数与方法签名不匹配导致的 TypeError。
                        err_msg = f"调用 Action '{function_name}' 时参数不匹配: {te}. 传入参数: {arguments}"
                        logger.error(f"[ToolExecutor] Action 调用参数错误: {err_msg}", exc_info=True)
                        action_result = {"status": "failure", "message": f"错误: 调用工具 '{function_name}' 时参数错误。", "error": {"type": "ArgumentMismatch", "details": err_msg}}
                        await async_print(f"  ❌ [{current_tool_index}/{total_tools}] 操作失败: {tool_display_name}. 错误: 调用参数错误。")
                    except Exception as exec_err:
                        # 捕获 Action 方法在执行过程中抛出的任何其他未预料到的异常。
                        err_msg = f"Action '{function_name}' 执行期间发生意外错误: {exec_err}"
                        logger.error(f"[ToolExecutor] Action 执行内部错误: {err_msg}", exc_info=True)
                        action_result = {"status": "failure", "message": f"错误: 执行工具 '{function_name}' 时发生内部错误。", "error": {"type": "ExecutionError", "details": str(exec_err)}}
                        await async_print(f"  ❌ [{current_tool_index}/{total_tools}] 操作失败: {tool_display_name}. 错误: 执行时内部错误。")
                else:
                    # 如果在 Agent 实例上找不到与 `function_name` 同名的方法。
                    err_msg = f"Agent 未实现名为 '{function_name}' 的工具方法。"
                    logger.error(f"[ToolExecutor] 工具未实现: {err_msg}")
                    action_result = {"status": "failure", "message": f"错误: {err_msg}", "error": {"type": "NotImplemented", "details": f"Action method '{function_name}' not found."}}
                    await async_print(f"  ❌ [{current_tool_index}/{total_tools}] 操作失败: {tool_display_name}. 错误: 工具未实现。")

            except Exception as outer_err:
                 # 这个 try-except 块捕获在处理单个工具调用的外层逻辑中发生的意外错误
                 # (例如，在解析 `mock_call` 结构之后、调用 Action 方法之前发生的错误)。
                 err_msg = f"处理工具调用 '{function_name}' (Mock ID: {tool_call_id}) 时发生顶层意外错误: {outer_err}"
                 logger.error(f"[ToolExecutor] 处理工具调用时顶层错误: {err_msg}", exc_info=True)
                 action_result = {"status": "failure", "message": f"错误: 处理工具 '{function_name}' 时发生未知内部错误。", "error": {"type": "Unexpected", "details": str(outer_err)}}
                 await async_print(f"  ❌ [{current_tool_index}/{total_tools}] 操作失败: {tool_display_name or function_name}. 错误: 未知内部错误。")


            # --- 4. 记录结果并检查是否需要中止 ---
            # 确保 `action_result` 变量总是被赋值 (根据代码逻辑，所有分支都应赋值)。
            if action_result is None:
                 # 这是一个内部逻辑错误的信号。
                 logger.error(f"[ToolExecutor] 内部逻辑错误: 工具 '{function_name}' (Mock ID: {tool_call_id}) 未生成任何结果。标记为失败。")
                 action_result = {"status": "failure", "message": f"错误: 工具 '{function_name}' 未返回结果。", "error": {"type": "MissingResult", "details": "Execution pipeline failed to produce a result."}}

            # 将当前工具的执行结果（包括 `status`, `message` 等）添加到结果列表中。
            execution_results.append({"tool_call_id": tool_call_id, "result": action_result})
            logger.debug(f"[ToolExecutor] 已记录工具 '{tool_call_id}' 的执行结果。")

            # 检查当前工具的执行状态，如果不是 'success'，则触发失败中止逻辑。
            if action_result.get("status") != "success":
                logger.warning(f"[ToolExecutor] 工具 '{function_name}' (Mock ID: {tool_call_id}) 执行失败 (状态: {action_result.get('status', 'failure')})。中止后续工具执行。")
                await async_print(f"  ⚠️ 由于工具 '{tool_display_name}' 执行失败，后续计划已中止。")
                break # 跳出 for 循环，不再处理列表中的剩余工具。

        total_executed = len(execution_results)
        logger.info(f"[ToolExecutor] 所有 {total_executed}/{total_tools} 个工具调用处理完毕 (可能因失败提前中止)。")
        return execution_results

# --- Agent 核心类 ---
class CircuitDesignAgent:
    """
    电路设计 Agent 核心实现。
    作为系统的中央协调器 (Orchestrator)，负责管理整个 Agent 的生命周期和工作流程。
    它编排了从接收用户请求到生成最终响应的完整过程，包括：
    感知用户输入 -> 更新记忆状态 -> 调用 LLM 进行规划 (生成行动计划) ->
    根据计划执行内部工具 (Action) -> 观察工具执行结果并更新记忆 ->
    再次调用 LLM 生成总结性的最终响应。
    利用 `asyncio` 实现高效的异步操作，并通过 `@register_tool` 装饰器动态管理可用的工具集。
    """
    def __init__(self, api_key: str, model_name: str = "glm-4-flash",
                 max_short_term_items: int = 20, max_long_term_items: int = 50,
                 planning_llm_retries: int = 1):
        logger.info(f"\n{'='*30} Agent 初始化开始 (Async, Decorator Tools) {'='*30}")
        logger.info("[Agent Init] 正在启动电路设计助理...")

        try:
            # 在初始化阶段，实例化并配置所有核心组件。
            self.memory_manager = MemoryManager(max_short_term_items, max_long_term_items)
            self.llm_interface = LLMInterface(api_key=api_key, model_name=model_name)
            self.output_parser = OutputParser()
            # ToolExecutor 需要传入 Agent 自身的实例 (`self`)，因为它需要回调 Agent 上定义的 Action 方法。
            self.tool_executor = ToolExecutor(agent_instance=self)
        except (ValueError, ConnectionError, TypeError) as e:
            # 核心组件的初始化失败是致命错误，记录日志并终止程序。
            logger.critical(f"[Agent Init] 核心模块初始化失败: {e}", exc_info=True)
            sys.stderr.write(f"\n🔴 Agent 核心模块初始化失败: {e}\n请检查配置或依赖！程序无法启动。\n")
            sys.stderr.flush()
            sys.exit(1) # 强制退出。

        # 配置规划阶段 LLM 调用的重试次数，增加系统对 LLM 不稳定性的容忍度。
        self.planning_llm_retries = max(0, planning_llm_retries) # 确保重试次数非负。
        logger.info(f"[Agent Init] 规划 LLM 调用失败时将重试 {self.planning_llm_retries} 次。")

        # --- 动态发现并注册工具 ---
        self.tools_registry: Dict[str, Dict[str, Any]] = {} # 用于存储已发现工具的名称及其 Schema。
        logger.info("[Agent Init] 正在动态发现并注册已标记的工具...")
        # 使用 `inspect.getmembers` 遍历当前实例 (`self`) 的所有方法。
        for name, method in inspect.getmembers(self, predicate=inspect.ismethod):
            # 检查方法是否带有由 `@register_tool` 装饰器设置的 `_is_tool` 标记。
            if hasattr(method, '_is_tool') and method._is_tool:
                # 如果是工具方法，尝试获取其附加的 Schema 信息。
                schema = getattr(method, '_tool_schema', None)
                if schema and isinstance(schema, dict):
                    # 验证 Schema 是否包含必需的 'description' 和 'parameters' 字段。
                    if 'description' in schema and 'parameters' in schema:
                        # 将有效的工具及其 Schema 添加到注册表中。
                        self.tools_registry[name] = schema
                        logger.info(f"[Agent Init] ✓ 已注册工具: '{name}'")
                    else:
                        # 如果 Schema 结构不完整，记录警告并跳过该工具。
                        logger.warning(f"[Agent Init] 发现工具 '{name}' 但其 Schema 结构不完整 (缺少 description 或 parameters)，已跳过。Schema: {schema}")
                else:
                    # 如果无法获取有效的 Schema，记录警告并跳过。
                    logger.warning(f"[Agent Init] 发现工具标记 '{name}' 但未能获取有效的 Schema，已跳过。")

        if not self.tools_registry:
            # 如果初始化后没有发现任何注册的工具，发出警告。
            logger.warning("[Agent Init] 未发现任何通过 @register_tool 注册的工具！Agent 将无法执行任何工具操作。")
        else:
            logger.info(f"[Agent Init] 共发现并注册了 {len(self.tools_registry)} 个工具。")
            # 打印详细的工具注册表信息，有助于调试和了解 Agent 的能力。
            logger.debug(f"[Agent Init] 工具注册表详情:\n{json.dumps(self.tools_registry, indent=2, ensure_ascii=False)}")

        logger.info(f"\n{'='*30} Agent 初始化成功 {'='*30}\n")
        print("我是电路设计编程助理！")
        print("已准备好接收指令。采用异步核心和动态工具注册。")
        print("-" * 70)
        sys.stdout.flush()


    # --- Action Implementations (Decorated & Standardized Output) ---
    # 以下是 Agent 可以执行的具体操作 (Action) 的实现。
    # 每个方法都使用了 `@register_tool` 装饰器来声明其功能和所需的参数。
    # 当前这些 Action 方法是同步定义的，它们将由 ToolExecutor 在独立的线程中异步执行。
    # 每个 Action 方法必须返回一个包含 `status` ('success' 或 'failure') 和 `message` 键的字典，
    # 以便 Orchestrator 和最终的 LLM 能够理解操作的执行结果。

    @register_tool(
        description="添加一个新的电路元件 (如电阻, 电容, 电池, LED, 开关, 芯片, 地线等)。如果用户未指定 ID，我会自动生成。元件值是可选的。",
        parameters={ # 参数 Schema 定义了此工具接受的输入。
            "type": "object",
            "properties": {
                "component_type": {"type": "string", "description": "元件的类型 (例如: '电阻', 'LED', '9V 电池')."},
                "component_id": {"type": "string", "description": "可选的用户指定 ID。如果省略会自动生成。请勿臆造不存在的 ID。"},
                "value": {"type": "string", "description": "可选的元件值 (例如: '1k', '10uF', '9V'). 如果未指定则省略。"}
            },
            "required": ["component_type"] # 表明 `component_type` 是必需参数。
        }
    )
    def add_component_tool(self, arguments: Dict[str, Any]) -> Dict[str, Any]:
        """
        Action 实现：添加一个电路元件。
        此方法负责：
        1. 验证输入参数的有效性 (特别是元件类型)。
        2. 处理元件 ID：如果用户提供了有效且未被占用的 ID，则使用该 ID；否则，调用 `MemoryManager` 自动生成一个唯一的 ID。
        3. 创建 `CircuitComponent` 对象实例。
        4. 将新创建的元件对象存储到 `MemoryManager` 的电路知识库中。
        5. 将此操作记录到长期记忆中，作为经验积累。
        6. 返回一个标准化的结果字典，包含操作状态 (`status`) 和描述信息 (`message`)。
        """
        logger.info("[Action: AddComponent] 执行添加元件操作。")
        logger.debug(f"[Action: AddComponent] 收到参数: {arguments}")
        component_type = arguments.get("component_type")
        component_id_req = arguments.get("component_id") # 用户请求的 ID (可能为 None)
        value = arguments.get("value") # 元件值 (可能为 None)
        logger.info(f"[Action: AddComponent] 参数解析: Type='{component_type}', Requested ID='{component_id_req}', Value='{value}'")

        # --- 参数基本验证 ---
        # 确保必需的 `component_type` 参数存在且有效。
        if not component_type or not isinstance(component_type, str) or not component_type.strip():
            msg="元件类型是必需的，并且必须是有效的字符串。"
            logger.error(f"[Action: AddComponent] 输入验证失败: {msg}")
            # 返回标准化的失败结果。
            return {"status": "failure", "message": f"错误: {msg}", "error": {"type": "InvalidInput", "details": msg}}

        target_id = None # 将用于存储最终确定的元件 ID。
        id_was_generated = False # 标记 ID 是否是自动生成的。
        user_provided_id_validated = None # 存储验证通过的用户指定 ID。

        # --- ID 处理逻辑 ---
        # 检查用户是否提供了非空的 ID 字符串。
        if component_id_req and isinstance(component_id_req, str) and component_id_req.strip():
            user_provided_id = component_id_req.strip().upper() # 清理并规范化用户提供的 ID。
            # 使用正则表达式验证 ID 格式是否符合基本要求 (例如，字母、数字、下划线、连字符)。
            # 这个规则可以根据实际需要调整。
            if re.match(r'^[a-zA-Z0-9_][a-zA-Z0-9_-]*$', user_provided_id):
                # 检查用户提供的 ID 是否已经在电路中被占用。
                if user_provided_id in self.memory_manager.circuit_knowledge["components"]:
                    msg=f"元件 ID '{user_provided_id}' 已被占用，请选择其他 ID。"
                    logger.error(f"[Action: AddComponent] ID 冲突: {msg}")
                    return {"status": "failure", "message": f"错误: {msg}", "error": {"type": "IDConflict", "details": msg}}
                else:
                    # 用户提供的 ID 有效且可用，将其设为目标 ID。
                    target_id = user_provided_id
                    user_provided_id_validated = target_id
                    logger.debug(f"[Action: AddComponent] 验证通过，将使用用户提供的 ID: '{target_id}'.")
            else:
                # 用户提供的 ID 格式无效，记录警告，并将触发自动生成逻辑。
                logger.warning(f"[Action: AddComponent] 用户提供的 ID '{component_id_req}' 格式无效 (允许字母、数字、下划线、连字符)，将自动生成 ID。")
                # `target_id` 保持为 None。
        else:
            # 用户未提供 ID 或提供了空字符串，需要自动生成。
            logger.debug("[Action: AddComponent] 用户未提供 ID 或提供的为空，将自动生成。")

        # --- 自动生成 ID (如果需要) ---
        # 如果 `target_id` 仍为 None (表示需要自动生成)。
        if target_id is None:
            try:
                # 调用 MemoryManager 的 ID 生成方法。
                target_id = self.memory_manager.generate_component_id(component_type)
                id_was_generated = True
                logger.debug(f"[Action: AddComponent] 已自动生成 ID: '{target_id}'.")
            except RuntimeError as e:
                # 捕获 ID 生成过程中可能发生的错误 (例如无法找到唯一 ID)。
                msg=f"无法自动为类型 '{component_type}' 生成唯一 ID: {e}"
                logger.error(f"[Action: AddComponent] ID 生成失败: {msg}", exc_info=True)
                return {"status": "failure", "message": f"错误: {msg}", "error": {"type": "IDGenerationFailed", "details": str(e)}}

        # --- 值处理 ---
        # 对元件值进行清理，确保是字符串或 None。
        processed_value = str(value).strip() if value is not None and str(value).strip() else None

        # --- 创建并存储元件对象 ---
        try:
            # 最后确认 `target_id` 已成功确定 (理论上此时不应为 None)。
            if target_id is None:
                # 这是一个内部逻辑错误的信号。
                raise ValueError("内部错误：未能最终确定元件 ID。")

            # 使用确定的 ID、类型和值创建 `CircuitComponent` 实例。
            new_component = CircuitComponent(target_id, component_type, processed_value)
            # 将新元件添加到 MemoryManager 的元件字典中。
            self.memory_manager.circuit_knowledge["components"][new_component.id] = new_component
            logger.info(f"[Action: AddComponent] 成功添加元件 '{new_component.id}' 到电路知识库。")

            # 构建用户友好的成功消息，并说明 ID 的来源。
            success_message = f"操作成功: 已添加元件 {str(new_component)}。"
            if id_was_generated:
                success_message += f" (系统自动分配 ID '{new_component.id}')"
            elif user_provided_id_validated:
                success_message += f" (使用了您指定的 ID '{user_provided_id_validated}')"

            # 将此成功操作记录到长期记忆。
            self.memory_manager.add_to_long_term(f"添加了元件: {str(new_component)}")

            # 返回标准化的成功结果字典，可以在 `data` 字段中包含额外信息 (如创建的元件详情)。
            return {
                "status": "success",
                "message": success_message,
                "data": {"id": new_component.id, "type": new_component.type, "value": new_component.value}
            }
        except ValueError as ve:
            # 捕获在创建 `CircuitComponent` 实例时可能发生的验证错误 (虽然理论上已被前置检查覆盖)。
            msg=f"创建元件对象时发生内部错误: {ve}"
            logger.error(f"[Action: AddComponent] 元件创建错误: {msg}", exc_info=True)
            return {"status": "failure", "message": f"错误: {msg}", "error": {"type": "ComponentCreationError", "details": str(ve)}}
        except Exception as e:
            # 捕获在存储或处理过程中发生的任何其他未预料到的错误。
            msg=f"添加元件时发生未知的内部错误: {e}"
            logger.error(f"[Action: AddComponent] 未知错误: {msg}", exc_info=True)
            return {"status": "failure", "message": "错误: 添加元件时发生未知内部错误。", "error": {"type": "Unexpected", "details": str(e)}}

    @register_tool(
        description="使用两个已存在元件的 ID 将它们连接起来。执行前我会检查元件是否存在。",
        parameters={
            "type": "object",
            "properties": {
                "comp1_id": {"type": "string", "description": "第一个元件的 ID (通常大写)。"},
                "comp2_id": {"type": "string", "description": "第二个元件的 ID (通常大写)。"}
            },
            "required": ["comp1_id", "comp2_id"] # 两个元件的 ID 都是必需的。
        }
    )
    def connect_components_tool(self, arguments: Dict[str, Any]) -> Dict[str, Any]:
        """
        Action 实现：在两个已存在的元件之间建立连接。
        此方法负责：
        1. 验证输入的两个 ID 是否有效。
        2. 检查具有这两个 ID 的元件是否都实际存在于当前的电路知识库中。
        3. 防止将一个元件连接到其自身。
        4. 检查该连接是否已经存在，避免重复添加。
        5. 如果所有检查通过，则将表示连接的元组 (按 ID 排序以保证唯一性) 添加到 `MemoryManager` 的 `connections` 集合中。
        6. 将操作记录到长期记忆。
        7. 返回标准化的结果字典。
        """
        logger.info("[Action: ConnectComponents] 执行连接元件操作。")
        logger.debug(f"[Action: ConnectComponents] 收到参数: {arguments}")
        comp1_id_req = arguments.get("comp1_id")
        comp2_id_req = arguments.get("comp2_id")
        logger.info(f"[Action: ConnectComponents] 参数解析: Comp1='{comp1_id_req}', Comp2='{comp2_id_req}'")

        # --- 输入验证 ---
        # 确保提供了两个有效的、非空的 ID 字符串。
        if not comp1_id_req or not isinstance(comp1_id_req, str) or not comp1_id_req.strip() or \
           not comp2_id_req or not isinstance(comp2_id_req, str) or not comp2_id_req.strip():
            msg="必须提供两个有效的、非空的元件 ID 字符串才能进行连接。"
            logger.error(f"[Action: ConnectComponents] 输入验证失败: {msg}")
            return {"status": "failure", "message": f"错误: {msg}", "error": {"type": "InvalidInput", "details": msg}}

        # 清理并规范化 ID (去除空格，转大写)。
        id1 = comp1_id_req.strip().upper()
        id2 = comp2_id_req.strip().upper()

        # 检查是否尝试将元件连接到自身。
        if id1 == id2:
            msg=f"不能将元件 '{id1}' 连接到它自己。"
            logger.error(f"[Action: ConnectComponents] 自连接错误: {msg}")
            return {"status": "failure", "message": f"错误: {msg}", "error": {"type": "SelfConnection", "details": msg}}

        # --- 检查元件是否存在 ---
        # 从 MemoryManager 获取当前的元件字典。
        components = self.memory_manager.circuit_knowledge["components"]
        # 检查第一个 ID 对应的元件是否存在。
        if id1 not in components:
            msg=f"无法连接：元件 '{id1}' 在当前电路中不存在。"
            logger.error(f"[Action: ConnectComponents] 元件不存在: {msg}")
            return {"status": "failure", "message": f"错误: {msg}", "error": {"type": "ComponentNotFound", "details": f"Component '{id1}' not found."}}
        # 检查第二个 ID 对应的元件是否存在。
        if id2 not in components:
            msg=f"无法连接：元件 '{id2}' 在当前电路中不存在。"
            logger.error(f"[Action: ConnectComponents] 元件不存在: {msg}")
            return {"status": "failure", "message": f"错误: {msg}", "error": {"type": "ComponentNotFound", "details": f"Component '{id2}' not found."}}

        # --- 执行连接 ---
        # 为了保证连接的唯一性并忽略顺序 (A-B 和 B-A 是同一个连接)，
        # 我们将两个 ID 排序后组成一个元组，并将其存储在集合 `connections` 中。
        # 集合的特性自动处理了重复添加的问题。
        connection = tuple(sorted((id1, id2)))
        connections_set = self.memory_manager.circuit_knowledge["connections"]

        # 检查这个连接是否已经存在于集合中。
        if connection in connections_set:
            msg=f"元件 '{id1}' 和 '{id2}' 之间已经存在连接。"
            logger.info(f"[Action: ConnectComponents] 连接已存在: {msg}")
            # 如果连接已存在，这通常不应视为失败，而是一个状态信息。返回成功状态，并在消息中说明。
            return {"status": "success", "message": f"注意: {msg}", "data": {"connection": list(connection)}}

        try:
            # 将新的连接元组添加到 `connections` 集合中。
            connections_set.add(connection)
            logger.info(f"[Action: ConnectComponents] 成功添加连接: {id1} <--> {id2}")
            # 将此成功操作记录到长期记忆。
            self.memory_manager.add_to_long_term(f"连接了元件: {id1} <--> {id2}")
            success_message = f"操作成功: 已将元件 '{id1}' 与 '{id2}' 连接起来。"
            # 返回标准化的成功结果。
            return {"status": "success", "message": success_message, "data": {"connection": list(connection)}}
        except Exception as e:
            # 捕获在添加连接到集合时可能发生的罕见错误。
            msg=f"添加连接时发生意外的内部错误: {e}"
            logger.error(f"[Action: ConnectComponents] 未知错误: {msg}", exc_info=True)
            return {"status": "failure", "message": "错误: 连接元件时发生未知内部错误。", "error": {"type": "Unexpected", "details": str(e)}}

    @register_tool(
        description="获取当前电路的详细描述，包括所有已添加的元件及其值（如果有）和所有连接。",
        parameters={"type": "object", "properties": {}} # 此工具不需要任何参数。
    )
    def describe_circuit_tool(self, arguments: Dict[str, Any]) -> Dict[str, Any]:
        """
        Action 实现：生成并返回当前电路状态的描述。
        它通过调用 `MemoryManager` 的 `get_circuit_state_description` 方法来获取格式化的文本描述。
        这个描述对于用户理解当前电路状态或供 LLM 在后续步骤中参考都非常有用。
        """
        logger.info("[Action: DescribeCircuit] 执行描述电路操作。")
        # 理论上此工具不接收参数，但仍记录以保持一致性。
        logger.debug(f"[Action: DescribeCircuit] 收到参数: {arguments} (应为空)")

        try:
            # 调用 MemoryManager 获取电路状态描述。
            description = self.memory_manager.get_circuit_state_description()
            logger.info("[Action: DescribeCircuit] 成功生成电路描述。")
            # 返回成功结果。将生成的描述文本放在 `data` 字段中，
            # 这样后续的 LLM 调用可以方便地使用这个信息。
            return {"status": "success", "message": "已成功获取当前电路的描述。", "data": {"description": description}}
        except Exception as e:
            # 捕获在生成描述过程中可能发生的任何错误。
            msg=f"生成电路描述时发生意外的内部错误: {e}"
            logger.error(f"[Action: DescribeCircuit] 未知错误: {msg}", exc_info=True)
            return {"status": "failure", "message": "错误: 获取电路描述时发生未知错误。", "error": {"type": "Unexpected", "details": str(e)}}

    @register_tool(
        description="彻底清空当前的电路设计，移除所有已添加的元件和它们之间的连接，并重置所有 ID 计数器。",
        parameters={"type": "object", "properties": {}} # 此工具也不需要任何参数。
    )
    def clear_circuit_tool(self, arguments: Dict[str, Any]) -> Dict[str, Any]:
        """
        Action 实现：清空整个电路状态。
        它通过调用 `MemoryManager` 的 `clear_circuit` 方法来执行实际的清空操作，
        将电路恢复到初始的空状态。
        """
        logger.info("[Action: ClearCircuit] 执行清空电路操作。")
        logger.debug(f"[Action: ClearCircuit] 收到参数: {arguments} (应为空)")

        try:
            # 调用 MemoryManager 执行清空操作。
            self.memory_manager.clear_circuit()
            logger.info("[Action: ClearCircuit] 电路状态已成功清空。")
            # 将此重要操作记录到长期记忆。
            self.memory_manager.add_to_long_term("执行了清空电路操作。")
            success_message = "操作成功: 当前电路已彻底清空。所有元件、连接和 ID 计数器均已重置。"
            # 返回标准化的成功结果。
            return {"status": "success", "message": success_message}
        except Exception as e:
            # 捕获在清空过程中可能发生的任何错误。
            msg=f"清空电路时发生意外的内部错误: {e}"
            logger.error(f"[Action: ClearCircuit] 未知错误: {msg}", exc_info=True)
            return {"status": "failure", "message": "错误: 清空电路时发生未知错误。", "error": {"type": "Unexpected", "details": str(e)}}


    # --- Orchestration Layer Method ---
    async def process_user_request(self, user_request: str) -> str:
        """
        处理用户请求的核心异步流程，实现了完整的 Agentic 循环。
        这是 Agent 的“大脑”或“指挥中心”，负责协调所有组件的工作：
        1.  **感知 (Perceive):** 接收用户输入，进行基本验证。
        2.  **记忆更新 (Memory Update):** 将用户输入添加到短期记忆。
        3.  **规划 (Plan):** 调用 LLM 生成执行计划 (自定义 JSON 格式)，包含重试逻辑。
        4.  **解析 (Parse):** 解析 LLM 返回的计划 JSON。
        5.  **决策 (Decide):** 根据计划决定是执行工具还是直接回复。
        6.  **行动 (Act):** 如果需要，调用 `ToolExecutor` 按顺序执行工具，处理失败中止。
        7.  **观察 (Observe):** 获取工具执行结果。
        8.  **记忆更新 (Memory Update):** 将工具执行结果添加到短期记忆。
        9.  **响应生成 (Respond):** (如果执行了工具) 再次调用 LLM，基于完整上下文生成最终的用户回复。
        10. **记忆更新 (Memory Update):** 将最终回复添加到短期记忆。
        11. **返回结果:** 将包含思考过程和正式回复的最终报告返回给调用者。
        """
        request_start_time = time.monotonic() # 记录请求处理开始时间，用于性能监控。
        logger.info(f"\n{'='*25} 开始处理用户请求 {'='*25}")
        logger.info(f"[Orchestrator] 收到用户指令: \"{user_request}\"")

        # --- 0. 输入验证 ---
        # 对用户输入进行基本的非空验证。
        if not user_request or user_request.isspace():
            logger.info("[Orchestrator] 用户指令为空或仅包含空白。")
            await async_print("\n您的指令似乎是空的，请重新输入！")
            # 对于无效输入，直接返回提示信息，无需启动完整的 Agent 循环。
            return "<think>用户输入为空或空白，无需处理。</think>\n\n请输入您的指令！"

        # --- 阶段 1: 感知与记忆更新 (用户输入) ---
        logger.info("--- [阶段 1] 感知与记忆更新 ---")
        try:
            # 将用户的请求作为 'user' 角色的消息添加到短期记忆中。
            # MemoryManager 会自动处理记忆修剪。
            self.memory_manager.add_to_short_term({"role": "user", "content": user_request})
            logger.info("[Orchestrator] 用户指令已记录并添加到短期记忆。")
        except Exception as e:
            # 如果添加到记忆失败，这是一个内部错误，需要通知用户并记录。
            logger.error(f"[Orchestrator] 添加用户消息到短期记忆时出错: {e}", exc_info=True)
            await async_print(f"\n🔴 抱歉，我在记录您的指令时遇到了内部问题 ({e})！请稍后重试。")
            # 返回包含错误信息的响应。
            return f"<think>添加用户消息到短期记忆失败: {e}</think>\n\n抱歉，我在处理您的指令时遇到了内部记忆错误，请稍后再试。"

        # --- 阶段 2: 规划 (第一次 LLM 调用 - 自定义 JSON 模式，带重试) ---
        logger.info("\n--- [阶段 2] 规划 (请求 LLM 生成执行计划) ---")
        await async_print("--- 正在请求智能大脑分析指令并生成执行计划 (JSON)... ---")

        # 准备传递给 LLM 的上下文信息。
        # 1. 获取当前的记忆上下文 (电路状态 + 近期长期记忆)。
        memory_context = self.memory_manager.get_memory_context_for_prompt()
        # 2. 获取当前所有可用工具的 Schema 描述字符串。
        tool_schemas_for_prompt = self._get_tool_schemas_for_prompt()
        if not self.tools_registry: # 如果没有注册工具，在 Prompt 中明确说明。
            logger.warning("[Orchestrator] 没有可用的工具。将在 Prompt 中说明。")

        # 3. 构建规划阶段专用的 System Prompt。
        system_prompt_planning = self._get_planning_prompt(tool_schemas_for_prompt, memory_context)

        # 4. 构建发送给 LLM 的完整消息列表。
        #    包含 System Prompt 和短期记忆中的所有非系统消息 (即完整的对话历史)。
        messages_for_llm1 = [{"role": "system", "content": system_prompt_planning}] + \
                           [msg for msg in self.memory_manager.short_term if msg.get("role") != "system"]

        # 初始化用于存储规划结果的相关变量。
        first_llm_response = None # 存储 LLM 的原始响应。
        thinking_process = "未能提取思考过程。" # 存储提取到的 <think> 内容。
        plan_dict = None # 存储最终解析并验证成功的 JSON 计划。
        parser_error_msg = "" # 存储解析或验证过程中发生的错误信息。
        attempt = 0 # 当前尝试次数计数器。
        max_attempts = 1 + self.planning_llm_retries # 总尝试次数 = 首次尝试 + 配置的重试次数。

        # --- LLM 调用与解析的重试循环 ---
        # 这个循环是为了处理 LLM 调用可能出现的瞬时失败 (如网络问题) 或返回格式错误 (如无效 JSON)。
        while attempt < max_attempts:
            attempt += 1
            logger.info(f"[Orchestrator] 尝试第 {attempt}/{max_attempts} 次调用规划 LLM...")
            await async_print(f"    (与大脑沟通尝试 {attempt}/{max_attempts})...")

            try:
                # **异步调用 LLM 进行规划**
                first_llm_response = await self.llm_interface.call_llm(
                    messages=messages_for_llm1,
                    use_tools=False # 明确指示 LLMInterface 不使用 SDK 的 tool calling 功能。
                )
                logger.info(f"[Orchestrator] 第 {attempt} 次 LLM 调用完成。")

                # --- 阶段 3: 解析自定义 JSON 规划响应 ---
                logger.info(f"--- [阶段 3 - 尝试 {attempt}] 解析 LLM 的规划响应 ---")
                # 对 LLM 响应进行基本有效性检查。
                if not first_llm_response or not first_llm_response.choices:
                     logger.error(f"[Orchestrator] 第 {attempt} 次 LLM 响应无效或缺少 'choices'。")
                     parser_error_msg = "LLM 响应无效或缺少 'choices'。"
                     # 如果响应结构本身就有问题，重试可能也无效，提前跳出循环。
                     break

                # 获取响应中的消息内容部分。
                response_message = first_llm_response.choices[0].message
                finish_reason = first_llm_response.choices[0].finish_reason # 获取完成原因。
                logger.info(f"[Orchestrator] 第 {attempt} 次 LLM 完成原因: '{finish_reason}'")
                raw_content_from_llm = getattr(response_message, 'content', None) # 获取原始文本内容。
                # 记录原始响应内容的一部分，便于调试 LLM 输出。
                logger.debug(f"[Orchestrator] 第 {attempt} 次 LLM 原始 Content (前 1000 字): >>>\n{str(raw_content_from_llm)[:1000]}...\n<<<")

                # 检查 LLM 是否意外返回了 `tool_calls` 字段 (这不符合我们的自定义 JSON 模式)。
                raw_tool_calls_from_llm = getattr(response_message, 'tool_calls', None)
                if raw_tool_calls_from_llm:
                    logger.warning(f"[Orchestrator] 警告 (尝试 {attempt})：LLM 在自定义 JSON 模式下意外返回了 'tool_calls' 字段！这将被忽略。")

                # **使用 OutputParser 解析并验证响应**
                thinking_process, plan_dict, parser_error_msg = self.output_parser.parse_planning_response(response_message)

                # 检查解析结果：`plan_dict` 不为 None 且 `parser_error_msg` 为空表示成功。
                if plan_dict is not None and not parser_error_msg:
                    # 解析和验证都成功！
                    logger.info(f"[Orchestrator] 第 {attempt} 次尝试成功解析并验证自定义 JSON 计划！")
                    break # 成功获取有效计划，跳出重试循环。
                else:
                    # 解析或验证失败。
                    logger.warning(f"[Orchestrator] 第 {attempt} 次尝试解析 JSON 失败: {parser_error_msg}")
                    # 如果还有重试次数，循环将继续；否则将在循环结束后处理此失败情况。

            except ConnectionError as conn_err:
                # 捕获 LLM 调用期间发生的连接或网络错误。
                logger.error(f"[Orchestrator] 第 {attempt} 次规划 LLM 调用失败 (连接错误): {conn_err}", exc_info=True)
                parser_error_msg = f"LLM 调用连接错误: {conn_err}"
                # 连接错误通常值得重试。
                if attempt >= max_attempts:
                    # 如果已达到最大尝试次数。
                    logger.critical(f"[Orchestrator] 所有 {max_attempts} 次规划尝试均因连接错误等失败。")
                    await async_print(f"\n🔴 抱歉，多次尝试连接智能大脑均失败 ({conn_err})！无法生成计划。")
                    # 返回包含详细错误信息的响应。
                    return f"<think>尝试 {max_attempts} 次规划 LLM 调用失败。最后错误: {parser_error_msg}\n最后思考过程: {thinking_process}</think>\n\n抱歉！我多次尝试联系智能大脑进行规划都失败了。请检查网络或 API 设置，或者稍后再试。"
                # else: 继续下一次重试 (循环会自动进行)。

            except Exception as e:
                # 捕获在 LLM 调用或响应解析过程中发生的其他所有未预料到的异常。
                logger.error(f"[Orchestrator] 第 {attempt} 次规划 LLM 调用或解析过程中发生严重错误: {e}", exc_info=True)
                parser_error_msg = f"LLM 调用或响应解析时发生错误: {e}"
                # 其他类型的异常 (例如解析逻辑错误) 可能重试无效，但仍然计入重试次数。
                if attempt >= max_attempts:
                    # 如果已达到最大尝试次数。
                    logger.critical(f"[Orchestrator] 所有 {max_attempts} 次规划尝试均失败。")
                    await async_print(f"\n🔴 抱歉，与智能大脑沟通时发生严重错误 ({e})！无法生成计划。")
                    # 返回包含详细错误信息的响应。
                    return f"<think>尝试 {max_attempts} 次规划 LLM 调用失败。最后错误: {parser_error_msg}\n最后思考过程: {thinking_process}</think>\n\n抱歉！在规划阶段遇到了严重的内部错误 ({e})。请检查日志或稍后再试。"
                # else: 继续下一次重试。

        # --- 重试循环结束 ---

        # 检查最终是否成功获取了有效的计划 (`plan_dict` 是否已赋值)。
        if plan_dict is None:
            # 如果 `plan_dict` 仍然是 None，说明所有尝试都失败了。
            logger.error(f"[Orchestrator] 经过 {attempt} 次尝试，未能成功获取有效的 JSON 计划。最终错误: {parser_error_msg}")
            await async_print(f"\n🔴 抱歉，我无法理解智能大脑生成的执行计划 ({parser_error_msg})！(尝试 {attempt} 次)")
            # 在最终的错误回复中包含更多上下文信息，帮助诊断问题。
            raw_content_snippet = f"\n最后一次 LLM 原始响应片段:\n{str(raw_content_from_llm)[:300]}..." if 'raw_content_from_llm' in locals() and raw_content_from_llm else ""
            return f"<think>解析 LLM 返回的自定义 JSON 失败 (尝试 {attempt} 次)。最终错误: {parser_error_msg}{raw_content_snippet}\n最后思考过程: {thinking_process}</think>\n\n抱歉！我尝试了 {attempt} 次，但还是无法解析智能大脑生成的执行计划。计划可能没有严格遵守格式要求，或者输出内容有问题。我已经记录了这个问题。"

        # --- 规划成功 ---
        logger.info("[Orchestrator] 成功获取并验证自定义 JSON 计划。")
        # 记录解析出的计划详情，便于调试。
        logger.debug(f"[Orchestrator] 解析出的计划详情: {json.dumps(plan_dict, indent=2, ensure_ascii=False)}")
        await async_print("--- 大脑已生成执行计划 ---")

        # --- 将 LLM 的规划响应添加到短期记忆 ---
        # 存储 LLM 返回的原始消息对象 (或其序列化表示)，以保持完整的对话历史记录。
        # MemoryManager 在添加时会处理可能的序列化和修剪。
        try:
             # 确保我们有有效的响应和消息对象。
             if first_llm_response and first_llm_response.choices:
                 assistant_plan_message = first_llm_response.choices[0].message
                 # 将 Pydantic 模型对象序列化为字典，以便存储在列表中。
                 # 使用 `.model_dump()` (Pydantic v2+) 或 `.dict()` (旧版)。
                 assistant_raw_response_for_memory = assistant_plan_message.model_dump(exclude_unset=True)
                 # 添加到短期记忆。
                 self.memory_manager.add_to_short_term(assistant_raw_response_for_memory)
                 logger.debug("[Orchestrator] LLM 的原始规划响应 (Message Dump) 已添加至短期记忆。")
             else:
                  # 这种情况理论上在 `plan_dict` 不为 None 时不应发生，但添加日志以防万一。
                  logger.error("[Orchestrator] 规划成功但无法获取有效的 LLM 响应对象以存入记忆。")
        except Exception as mem_err:
             # 记录添加记忆失败的错误，但尝试继续执行后续步骤。
             logger.error(f"[Orchestrator] 添加 LLM 规划响应到短期记忆失败: {mem_err}", exc_info=True)

        # --- 决策：根据计划执行工具或直接回复 ---
        # 从验证后的计划中提取关键决策信息。
        is_tool_calls = plan_dict.get("is_tool_calls", False) # 是否需要调用工具？
        tool_list_from_plan = plan_dict.get("tool_list") # 工具调用列表 (如果 is_tool_calls=true)。
        direct_reply_from_plan = plan_dict.get("direct_reply") # 直接回复内容 (如果 is_tool_calls=false)。

        # --- 情况 A: 计划要求执行工具 ---
        if is_tool_calls:
            logger.info("[Orchestrator] 决策：根据 JSON 计划执行工具。")

            # 进行防御性编程：再次验证 `tool_list` 是否为有效的非空列表。
            # 尽管 OutputParser 已经验证过，但多一层检查更安全。
            if not isinstance(tool_list_from_plan, list) or not tool_list_from_plan:
                 logger.error("[Orchestrator] 规划错误: 'is_tool_calls' 为 true 但 'tool_list' 不是有效的非空列表！")
                 await async_print("\n🔴 内部错误：执行计划要求调用工具，但工具列表无效。")
                 # 返回包含错误信息的响应。
                 return f"<think>{thinking_process}\n规划错误：指示调用工具，但 tool_list 无效或为空。Plan: {plan_dict}</think>\n\n抱歉，执行计划似乎存在内部问题（缺少有效的工具列表）。请重试或联系技术支持。"

            # --- 排序工具列表 ---
            # 根据 LLM 在 JSON 计划中指定的 `index` 字段对工具调用进行排序，
            # 确保严格按照计划的顺序执行。
            try:
                sorted_tool_list = sorted(tool_list_from_plan, key=lambda x: x.get('index', float('inf')))
                # 可选：检查索引是否从 1 开始且连续，记录警告（但不阻止执行）。
                expected_indices = list(range(1, len(sorted_tool_list) + 1))
                actual_indices = [item.get('index') for item in sorted_tool_list]
                if actual_indices != expected_indices:
                     logger.warning(f"[Orchestrator] 注意: 工具列表索引不规范 (应为 {expected_indices}, 实际为 {actual_indices})。已按 index 排序执行。")
                logger.info(f"[Orchestrator] 工具列表已按 index 排序，共 {len(sorted_tool_list)} 个工具需要执行。")
                logger.debug(f"[Orchestrator] 排序后的工具列表: {json.dumps(sorted_tool_list, indent=2, ensure_ascii=False)}")
            except Exception as sort_err:
                 # 排序失败通常是内部逻辑错误。
                 logger.error(f"[Orchestrator] 排序工具列表时出错: {sort_err}", exc_info=True)
                 await async_print("\n🔴 抱歉，我在安排工具执行顺序时遇到了内部问题。")
                 return f"<think>{thinking_process}\n排序工具列表时出错: {sort_err}</think>\n\n抱歉，我在处理工具执行顺序时遇到了内部错误。"

            # --- 将自定义工具列表转换为模拟 ToolCall 列表 ---
            # ToolExecutor 被设计为接收类似 OpenAI ToolCall 格式的对象列表。
            # 因此，我们需要将从自定义 JSON 计划中解析出的 `tool_list` 转换为这种格式。
            mock_tool_calls_for_executor = []
            conversion_successful = True
            for tool_item in sorted_tool_list:
                tool_name = tool_item.get("toolname")
                params_dict = tool_item.get("params", {}) # 获取参数字典
                index = tool_item.get("index")
                # 为每个模拟调用生成一个唯一的、包含信息的 ID，便于后续追踪和关联结果。
                # 使用参数的哈希值可以增加 ID 的独特性。
                params_hash = hash(json.dumps(params_dict, sort_keys=True)) & 0xffff # 取哈希值的低 16 位
                mock_id = f"call_{index}_{tool_name[:8]}_{params_hash:x}" # 格式: call_序号_工具名缩写_哈希值

                try:
                    # 将参数字典序列化回 JSON 字符串，因为 ToolExecutor 需要这种格式。
                    params_str = json.dumps(params_dict)
                except TypeError as json_dump_err:
                    # 如果参数字典中包含无法序列化为 JSON 的对象，记录错误并使用空 JSON 对象作为回退。
                    logger.error(f"转换工具 {tool_name} (index {index}) 的参数字典为 JSON 字符串失败: {json_dump_err}. Params: {params_dict}", exc_info=True)
                    conversion_successful = False
                    params_str = "{}" # 回退为空 JSON 对象字符串。

                # 构建模拟的 ToolCall 对象结构。
                mock_call = {
                    "id": mock_id, # 使用生成的唯一 ID。
                    "type": "function", # 假设所有工具都模拟为 'function' 类型。
                    "function": {"name": tool_name, "arguments": params_str}
                }
                mock_tool_calls_for_executor.append(mock_call)

            if not conversion_successful:
                 logger.warning("[Orchestrator] 注意: 转换自定义工具列表时遇到参数序列化问题。后续执行可能受影响。")
            logger.info(f"[Orchestrator] 成功将自定义工具列表转换为 {len(mock_tool_calls_for_executor)} 个模拟 ToolCall 对象，准备执行。")
            logger.debug(f"[Orchestrator] 转换后的模拟 ToolCall 列表: {json.dumps(mock_tool_calls_for_executor, indent=2)}")


            # --- 阶段 4: 行动执行 (调用 ToolExecutor - 异步 & 失败中止) ---
            logger.info("\n--- [阶段 4] 行动 (执行工具) ---")
            num_tools_to_run = len(mock_tool_calls_for_executor)
            await async_print(f"--- 正在按计划执行 {num_tools_to_run} 个操作 (若有失败则中止)... ---")

            tool_execution_results = [] # 用于存储 ToolExecutor 返回的实际执行结果。
            try:
                # **异步调用 ToolExecutor 执行工具**
                # ToolExecutor 会按顺序处理 `mock_tool_calls_for_executor` 列表，
                # 并在遇到第一个失败时停止。
                tool_execution_results = await self.tool_executor.execute_tool_calls(mock_tool_calls_for_executor)
                num_actually_executed = len(tool_execution_results) # 获取实际执行的工具数量。
                logger.info(f"[Orchestrator] ToolExecutor 完成了 {num_actually_executed}/{num_tools_to_run} 个工具执行尝试。")
                # 检查是否有工具因为失败中止而被跳过。
                if num_actually_executed < num_tools_to_run:
                     logger.warning(f"[Orchestrator] 由于中途有工具失败，计划中的后续 {num_tools_to_run - num_actually_executed} 个工具未执行。")
                await async_print(f"--- {num_actually_executed}/{num_tools_to_run} 个操作已执行 ---")
            except Exception as e:
                 # 捕获 ToolExecutor 本身执行过程中可能抛出的顶层意外错误
                 # (理论上 ToolExecutor 内部有错误处理，这种情况应很少见)。
                 logger.error(f"[Orchestrator] ToolExecutor 执行过程中发生顶层意外错误: {e}", exc_info=True)
                 await async_print(f"\n🔴 抱歉，执行工具时系统发生严重错误 ({e})！")
                 # 即使 ToolExecutor 本身出错，也尝试构造一个包含错误信息的失败结果，
                 # 以便后续流程能了解到发生了问题。
                 tool_execution_results.append({
                     "tool_call_id": "executor_error", # 特殊 ID 表示是执行器层面的错误。
                     "result": {"status": "failure", "message": f"错误: 工具执行器层面发生严重错误: {e}", "error": {"type": "ExecutorError", "details": str(e)}}
                 })


            # --- 阶段 5: 观察与记忆更新 (工具结果) ---
            logger.info("\n--- [阶段 5] 观察 (处理工具结果并更新记忆) ---")
            num_tool_results_added = 0
            if not tool_execution_results:
                # 如果 ToolExecutor 没有返回任何结果 (例如输入的列表为空，或第一个工具就失败了)。
                logger.warning("[Orchestrator] ToolExecutor 未返回任何执行结果。")
            else:
                # 将每个工具的执行结果格式化为 'tool' 角色的消息，并添加到短期记忆中。
                # 这为后续生成最终响应的 LLM 提供了关键的上下文信息。
                for exec_result in tool_execution_results:
                    # 使用模拟调用时生成的 `tool_call_id` 来关联结果。
                    tool_call_id_for_memory = exec_result.get('tool_call_id', 'unknown_mock_id')
                    # 获取结果字典，并提供默认值以防数据丢失。
                    result_dict = exec_result.get('result', {"status": "unknown", "message": "执行结果丢失"})
                    # 确保 `result_dict` 是一个字典，以便序列化。
                    if not isinstance(result_dict, dict):
                        logger.warning(f"工具 {tool_call_id_for_memory} 的结果不是字典格式，尝试包装。原始结果: {result_dict}")
                        result_dict = {"status": "unknown", "message": "非字典格式的工具结果", "raw_result": str(result_dict)}

                    try:
                        # 将结果字典转换为格式化的 JSON 字符串，存储在消息的 `content` 字段中。
                        # 使用 `default=str` 来处理可能包含无法直接序列化的类型 (例如异常对象) 的情况。
                        result_content_str = json.dumps(result_dict, indent=2, ensure_ascii=False, default=str)
                    except Exception as json_dump_error:
                        # 如果序列化失败，记录错误并生成一个包含错误信息的备用字符串。
                        logger.error(f"序列化工具 {tool_call_id_for_memory} 的结果字典失败: {json_dump_error}. Result: {result_dict}")
                        result_content_str = f'{{"status": "serialization_error", "message": "Failed to serialize result dict: {json_dump_error}", "original_result_repr": "{repr(result_dict)[:100]}..."}}'

                    # 构建符合 'tool' 角色规范的消息字典。
                    tool_message = {"role": "tool", "tool_call_id": tool_call_id_for_memory, "content": result_content_str}
                    try:
                        # 将 'tool' 消息添加到短期记忆。
                        self.memory_manager.add_to_short_term(tool_message)
                        num_tool_results_added += 1
                    except Exception as mem_err:
                        # 记录添加记忆失败，但继续处理其他结果。
                        logger.error(f"添加工具 {tool_call_id_for_memory} 结果到短期记忆失败: {mem_err}", exc_info=True)

            logger.info(f"[Orchestrator] {num_tool_results_added}/{len(tool_execution_results)} 个工具执行结果已添加至短期记忆。")
            # 在工具执行后，打印更新后的电路状态，这对于调试非常有帮助。
            logger.debug("工具执行后的电路状态:\n%s", self.memory_manager.get_circuit_state_description())


            # --- 阶段 6: 回复生成 (第二次 LLM 调用 - 异步) ---
            logger.info("\n--- [阶段 6] 响应生成 (请求 LLM 总结结果) ---")
            await async_print("--- 正在请求智能大脑总结操作结果并生成最终报告... ---")

            # 准备第二次 LLM 调用的上下文。
            # 1. 获取更新后的记忆上下文 (可能包含了新的电路状态)。
            memory_context_final = self.memory_manager.get_memory_context_for_prompt()
            # 2. 获取工具 Schema 描述 (供 LLM 参考，即使它不应再调用工具)。
            tool_schemas_final = self._get_tool_schemas_for_prompt()
            # 3. 确定是否有工具因为失败中止而被跳过。
            tools_were_skipped = num_actually_executed < num_tools_to_run
            # 4. 构建用于响应生成的 System Prompt。
            system_prompt_response_generation = self._get_response_generation_prompt(
                memory_context_final,
                tool_schemas_final,
                tools_were_skipped # 将是否有工具被跳过的信息传递给 Prompt。
            )

            # 5. 构建第二次 LLM 调用的完整消息列表。
            #    这次需要包含短期记忆中的 *所有* 消息：
            #    System Prompt, User 请求, Assistant 的规划响应, 以及所有 Tool 执行结果。
            messages_for_llm2 = [{"role": "system", "content": system_prompt_response_generation}] + \
                               self.memory_manager.short_term # 包含了完整的交互历史。

            # 初始化用于存储最终响应的变量。
            second_llm_response = None
            final_report = "" # 存储最终返回给用户的完整报告字符串 (<think> + reply)。
            try:
                 logger.info("[Orchestrator] 准备执行第二次 LLM 调用 (用于生成最终响应)...")
                 # **异步调用 LLM 生成最终回复**
                 second_llm_response = await self.llm_interface.call_llm(
                     messages=messages_for_llm2,
                     use_tools=False # 明确禁止 LLM 在生成最终回复时再次尝试调用工具。
                 )
                 logger.info("[Orchestrator] 第二次 LLM 调用完成。")
                 await async_print("--- 大脑已生成最终报告 ---")

                 # --- 阶段 7: 解析最终报告 ---
                 logger.info("\n--- [阶段 7] 解析最终报告 ---")
                 # 处理 LLM 可能返回无效响应或空内容的情况。
                 if not second_llm_response or not second_llm_response.choices or not second_llm_response.choices[0].message or not second_llm_response.choices[0].message.content:
                     logger.error("[Orchestrator] 第二次 LLM 响应无效或内容为空。无法生成最终报告。")
                     # 生成一个备用的错误报告。
                     final_thinking = "第二次 LLM 响应无效或内容为空。"
                     final_reply = "抱歉，我在总结结果时遇到了问题，智能大脑没有返回有效的报告内容。请参考之前的工具执行日志了解详情。"
                     # 尝试将这个错误情况作为 assistant 消息添加到记忆中。
                     try: self.memory_manager.add_to_short_term({"role": "assistant", "content": f"<think>{final_thinking}</think>\n\n{final_reply}"})
                     except Exception: pass # 忽略添加记忆可能发生的错误。
                     await async_print("\n🔴 抱歉，大脑未能生成最终报告！")
                 else:
                     # 获取最终的响应消息对象。
                     final_response_message = second_llm_response.choices[0].message
                     logger.info(f"[Orchestrator] 第二次 LLM 完成原因: '{second_llm_response.choices[0].finish_reason}'")
                     raw_final_content = final_response_message.content # 获取原始文本内容。
                     # 记录最终响应的原始内容（截断），用于调试。
                     logger.debug(f"[Orchestrator] 第二次 LLM 原始 Content (前 1000 字): >>>\n{str(raw_final_content)[:1000]}...\n<<<")

                     # 使用 OutputParser 解析最终的 `<think>` 思考过程和正式回复文本。
                     final_thinking, final_reply = self.output_parser._parse_llm_text_content(raw_final_content)

                     # 将最终的 assistant 回复（原始消息对象的 dump）添加到短期记忆，完成本次交互记录。
                     try:
                         final_assistant_message_dump = final_response_message.model_dump(exclude_unset=True)
                         self.memory_manager.add_to_short_term(final_assistant_message_dump)
                         logger.debug("[Orchestrator] 最终 LLM 回复 (原始 Message Dump) 已添加至短期记忆。")
                     except Exception as mem_err:
                         logger.error(f"添加最终回复到短期记忆失败: {mem_err}", exc_info=True)

                 # 组合最终返回给用户的报告字符串（包含思考过程和正式回复）。
                 final_report = f"<think>{final_thinking}</think>\n\n{final_reply}".rstrip()

            except Exception as e:
                 # 捕获在第二次 LLM 调用或最终报告处理过程中发生的任何其他错误。
                 logger.critical(f"[Orchestrator] 第二次 LLM 调用或最终报告处理失败: {e}", exc_info=True)
                 # 生成一个备用的错误报告。
                 fallback_thinking = f"第二次 LLM 调用或最终报告处理失败: {e}"
                 fallback_reply = f"抱歉，在为您准备最终报告时遇到了严重的内部错误 ({e})！请参考日志获取技术详情。"
                 # 尝试将此错误记录到记忆中。
                 try: self.memory_manager.add_to_short_term({"role": "assistant", "content": f"<think>{fallback_thinking}</think>\n\n{fallback_reply}"})
                 except Exception: pass
                 final_report = f"<think>{fallback_thinking}</think>\n\n{fallback_reply}".rstrip()
                 await async_print(f"\n🔴 抱歉，生成最终报告时发生严重错误 ({e})！")

            # 请求处理结束（工具调用路径）。
            request_end_time = time.monotonic()
            logger.info(f"\n{'='*25} 请求处理完毕 (工具调用路径, 耗时: {request_end_time - request_start_time:.3f} 秒) {'='*25}\n")
            return final_report

        # --- 情况 B: 计划不需要执行工具，直接回复 ---
        else: # is_tool_calls is False
            logger.info("[Orchestrator] 决策：根据 JSON 计划直接回复，不执行工具。")
            await async_print("--- 大脑认为无需执行操作，将直接回复... ---")

            # 直接使用第一次 LLM 调用（规划阶段）生成的 `direct_reply` 作为最终回复。
            # 需要再次检查 `direct_reply` 是否有效 (虽然 OutputParser 已验证)。
            if direct_reply_from_plan and isinstance(direct_reply_from_plan, str) and direct_reply_from_plan.strip():
                logger.info("[Orchestrator] 使用计划中提供的 'direct_reply' 作为最终回复。")
                # 复用第一次 LLM 调用提取的思考过程。
                final_thinking = thinking_process
                final_reply = direct_reply_from_plan
                # 在这种情况下，没有第二次 LLM 调用。
            else:
                # 如果 `is_tool_calls` 为 false，但 `direct_reply` 无效或缺失，这是一个规划错误。
                logger.error("[Orchestrator] 规划错误: 'is_tool_calls' 为 false 但 'direct_reply' 无效或缺失！")
                await async_print("\n🔴 内部错误：计划无需操作，但大脑没有提供回复内容！")
                # 生成一个包含错误信息的回复。
                final_thinking = thinking_process + "\n规划错误：is_tool_calls 为 false 但 direct_reply 无效或缺失。"
                final_reply = "我理解现在不需要执行操作，但是智能大脑没有提供相应的回复。这可能是一个规划错误，请您澄清指令或重试。"

            # 组合最终报告。
            final_report = f"<think>{final_thinking}</think>\n\n{final_reply}".rstrip()
            # 注意：第一次 LLM 的规划响应（其中包含了这个 `direct_reply`）已经在前面被添加到记忆中了。

            # 请求处理结束（直接回复路径）。
            request_end_time = time.monotonic()
            logger.info(f"\n{'='*25} 请求处理完毕 (直接回复路径, 耗时: {request_end_time - request_start_time:.3f} 秒) {'='*25}\n")
            return final_report

    # --- Helper Methods for Prompts ---
    def _get_tool_schemas_for_prompt(self) -> str:
        """
        根据 `self.tools_registry` 中动态发现和注册的工具信息，
        生成一段格式化的文本描述，用于注入到 LLM 的 Prompt 中。
        这种动态生成的方式避免了在 Prompt 模板中硬编码工具列表，提高了灵活性和可维护性。
        """
        if not self.tools_registry:
            # 如果没有注册任何工具，则明确告知 LLM。
            return "  (无可用工具)"

        tool_schemas = []
        # 遍历注册表中的每个工具及其 Schema。
        for name, schema in self.tools_registry.items():
            desc = schema.get('description', '无描述。') # 获取工具的自然语言描述。
            params = schema.get('parameters', {}) # 获取参数 Schema。
            props = params.get('properties', {}) # 获取具体的参数属性定义。
            req = params.get('required', []) # 获取必需参数的列表。
            param_desc_parts = []
            if props:
                # 格式化每个参数的描述信息。
                for k, v in props.items():
                    p_type = v.get('type', 'any') # 参数类型。
                    p_desc = v.get('description', '') # 参数描述。
                    # 标记参数是否必需。
                    p_req = '(必须)' if k in req else '(可选)'
                    param_desc_parts.append(f"{k}: {p_type} {p_req} '{p_desc}'")
                param_desc_str = "; ".join(param_desc_parts) # 使用分号连接所有参数的描述。
            else:
                param_desc_str = "无参数" # 如果工具没有参数。
            # 格式化单个工具的完整描述，包括名称、功能描述和参数信息。
            tool_schemas.append(f"  - `{name}`: {desc} (参数: {param_desc_str})")
        # 将所有工具的描述合并为一个多行字符串。
        return "\n".join(tool_schemas)

    def _get_planning_prompt(self, tool_schemas_desc: str, memory_context: str) -> str:
        """
        构建用于第一次规划调用的 System Prompt。
        这个 Prompt 的核心要求是指导 LLM：
        1.  分析用户请求、对话历史和当前状态。
        2.  决定是否需要调用工具以及调用哪些工具。
        3.  严格按照 `<think>...</think> JSON_OBJECT` 的格式输出其思考过程和最终计划。
        4.  JSON 对象必须符合我们定义的严格 Schema。
        5.  明确禁止 LLM 使用其内置的 Function Calling 功能或生成 `tool_calls` 字段。
        """
        return (
            "你是一位顶尖的、极其严谨的电路设计编程助理。你的行为必须专业、精确，并严格遵循指令。\n"
            "你的任务是：分析用户的最新指令、对话历史以及当前的电路状态，然后**严格按照下面描述的固定格式**生成一个包含执行计划的 JSON 对象。\n"
            "**绝对禁止**使用任何形式的 Function Calling 或生成 `tool_calls` 字段。你的**唯一输出**必须由两部分组成：\n"
            "1.  一个 `<think>...</think>` XML 块：在其中详细阐述你的思考过程。这应包括：对用户指令的理解，对当前电路状态和记忆的分析，决定是否需要调用工具以及调用哪些工具，如何从指令中提取参数，规划具体的执行步骤，以及对潜在问题的评估。\n"
            "2.  **紧随其后**，**必须**是一个**单一的、格式完全正确的 JSON 对象**：这个 JSON 对象代表了你最终的执行计划或直接回复。**不允许在 JSON 对象的前面或后面添加任何额外的文字、解释或注释！**\n\n"
            "**JSON 对象格式规范 (必须严格遵守):**\n"
            "该 JSON 对象**必须**包含以下字段：\n"
            "  - `is_tool_calls` (boolean): **必须字段**。如果分析后认为需要执行一个或多个工具操作来满足用户请求，则设为 `true`。如果不需要执行任何工具（例如，可以直接回答问题、进行确认、或者认为请求无法处理），则设为 `false`。\n"
            "  - `tool_list` (array<object> | null): **必须字段**。行为取决于 `is_tool_calls` 的值：\n"
            "     - 当 `is_tool_calls` 为 `true` 时: **必须**是一个包含**一个或多个**工具调用对象的数组。数组中的对象**必须按照你期望的执行顺序列出**。\n"
            "     - 当 `is_tool_calls` 为 `false` 时: 此字段**必须**是 `null` 值或者一个空数组 `[]`。\n"
            "     每个工具调用对象（如果存在）**必须**包含以下字段：\n"
            "       - `toolname` (string): **必须**。要调用的工具的**精确名称**。你必须从下面提供的“可用工具列表”中选择一个有效的名称。\n"
            "       - `params` (object): **必须**。一个包含调用该工具所需参数的 JSON 对象。你必须**严格**根据该工具的参数规范，从用户指令或对话历史中提取参数值。如果某个工具不需要参数，则提供一个空对象 `{}`。\n"
            "       - `index` (integer): **必须**。表示此工具调用在你本次规划中的执行顺序。**必须**是一个从 `1` 开始的正整数。如果本次规划包含多个工具调用，它们的 `index` 值必须是连续的（例如 1, 2, 3）。\n"
            "  - `direct_reply` (string | null): **必须字段**。行为取决于 `is_tool_calls` 的值：\n"
            "     - 当 `is_tool_calls` 为 `false` 时: 这里**必须**包含你准备直接回复给用户的最终、完整、友好的文本内容。回复内容不能为空字符串。\n"
            "     - 当 `is_tool_calls` 为 `true` 时: 此字段**必须**是 `null` 值。（因为后续会通过执行工具并再次调用你来生成最终回复）。\n\n"
            "**可用工具列表与参数规范:**\n"
            # 动态生成的工具描述字符串在此处注入。
            f"{tool_schemas_desc}\n\n"
            "**当前电路状态与记忆:**\n"
            # 格式化的记忆上下文在此处注入。
            f"{memory_context}\n\n"
            "**最后再次强调：你的回复格式必须严格是 `<think>思考过程</think>` 后面紧跟着一个符合上述规范的 JSON 对象。不允许有任何偏差！** JSON 的语法（括号、引号、逗号、数据类型）和结构（必需字段、条件字段）都必须完全正确，否则后续处理会失败。"
        )

    def _get_response_generation_prompt(self, memory_context: str, tool_schemas_desc: str, tools_were_skipped: bool) -> str:
        """
        构建用于第二次响应生成调用的 System Prompt。
        这个调用的上下文包含了用户请求、Agent 的规划、以及所有已执行工具的结果。
        此 Prompt 的核心任务是指导 LLM：
        1.  理解完整的对话历史，特别是 'tool' 角色消息中包含的工具执行结果 (`status`, `message`)。
        2.  基于这些信息，生成一个最终的、面向用户的、总结性的回复。
        3.  如果 `tools_were_skipped` 为 true，需要在报告中解释哪些计划步骤未执行及其原因。
        4.  同样要求以 `<think>...</think>\n\n正式回复` 的格式输出。
        5.  明确禁止在此阶段再次生成工具调用或 JSON 对象。
        """
        # 根据是否有工具被跳过，动态生成提示信息。
        skipped_info = ""
        if tools_were_skipped:
            skipped_info = "\n**重要提示：** 在之前的工具执行过程中，由于某个工具失败，后续计划中的一个或多个工具可能已被跳过执行。请在你的最终报告中明确说明这一点，解释哪些任务（如果有的话）因此未能完成，并总结当前任务的最终状态。"

        return (
            "你是一位顶尖的电路设计编程助理，经验丰富，技术精湛，并且擅长清晰地汇报工作结果。\n"
            "你的当前任务是：基于到目前为止的完整对话历史（包括用户最初的指令、你之前的思考和规划、以及所有已执行工具的结果），生成最终的、面向用户的文本回复。\n"
            "**关键信息来源是角色为 'tool' 的消息**: 每条 'tool' 消息都对应一个之前执行的工具调用（通过 `tool_call_id` 关联）。其 `content` 字段是一个 JSON 字符串，包含了该工具执行的关键信息，特别是 `status` 字段（指示 'success' 或 'failure'）和 `message` 字段（描述结果或错误）。可能还包含 `error` 字段提供失败的详细技术信息。\n"
            "**你的报告必须：**\n"
            "1.  仔细阅读并理解所有历史消息，**特别是要解析每条 'tool' 消息中的 JSON 内容**，准确把握每个已执行工具的**最终状态 (`status`)** 和结果 (`message`)。\n"
            "2.  清晰地向用户总结所有**已执行**工具操作的结果：\n"
            "    - 对于成功的操作 (`status: \"success\"`)，进行简要的确认。\n"
            "    - 对于失败的操作 (`status: \"failure\"`)，必须诚恳地向用户说明操作失败了，并根据 `message` 和可能的 `error` 字段解释失败的原因及其对整体任务的影响。\n"
            f"{skipped_info}\n" # 如果有工具被跳过，相关的提示信息会在这里插入。
            "3.  综合以上信息，回答用户最初的问题或确认任务的完成情况。如果任务因工具失败而未能完全完成，请明确说明当前的状态。\n"
            "4.  **严格按照以下固定格式**生成你的回复：\n"
            "   a. **思考过程:** 首先，在 `<think>` 和 `</think>` 标签之间，详细阐述你的反思和报告组织思路。回顾用户的原始请求、你的规划、并**重点分析所有已执行工具的 `status` 和 `message`**。评估任务的整体完成度。如果 `tools_were_skipped` 为真，要考虑未执行步骤的影响。最后，规划如何将这些信息整合，组织成清晰、友好的最终回复。\n"
            "   b. **正式回复:** 在 `</think>` 标签之后，紧跟着面向用户的正式文本回复。这个回复应该基于你的思考过程，清晰、简洁、友好地总结情况。\n"
            "**最终输出格式必须严格是:**\n"
            "`<think>你的详细思考过程</think>\n\n你的正式回复文本`\n"
            "(注意：`</think>` 标签后必须恰好是两个换行符 `\\n\\n`，然后直接是正式回复文本。)\n"
            "**重要：** 在这个阶段，你**绝对不能**再生成任何工具调用或 JSON 对象。你的唯一输出应该是包含 `<think>` 块和正式回复文本的字符串。"
            "\n\n"
            "**上下文参考信息:**\n"
            "【当前电路状态与记忆】\n"
            f"{memory_context}\n" # 注入最新的记忆上下文。
            "【我的可用工具列表 (仅供你参考，不应再次调用)】\n"
            f"{tool_schemas_desc}\n" # 提供工具列表供参考。
        )


# --- 异步主函数 (应用程序入口) ---
async def main():
    """
    应用程序的主入口点和异步执行流程。
    负责初始化 Agent，处理 API Key 获取，并启动主交互循环来接收和处理用户输入。
    """
    # 使用异步打印函数输出启动信息。
    await async_print("=" * 70)
    await async_print("🚀 启动 OpenManus 电路设计 Agent 🚀")
    await async_print("   特性: 异步核心, 动态工具注册, 规划重试, 失败中止, 内存修剪")
    await async_print("=" * 70)
    logger.info("[Main] 开始 Agent 初始化...")

    # --- 获取 API Key ---
    # 优先尝试从环境变量 `ZHIPUAI_API_KEY` 读取 API Key，这是更安全的做法。
    api_key = os.environ.get("ZHIPUAI_API_KEY")
    if not api_key:
        # 如果环境变量未设置，则提示用户手动输入。
        logger.warning("[Main] 环境变量 ZHIPUAI_API_KEY 未设置。")
        await async_print("\n为了连接智能大脑，我需要您的智谱AI API Key。")
        try:
            # 注意：标准 `input()` 是同步阻塞调用。在简单的命令行应用中可以接受。
            # 对于需要更高并发或非阻塞输入的场景 (如 GUI, Web 服务器)，应使用相应的异步 I/O 库。
            api_key = input("👉 请在此输入您的智谱AI API Key: ").strip()
        except (EOFError, KeyboardInterrupt):
            # 处理用户中断输入的情况 (例如按下 Ctrl+D 或 Ctrl+C)。
            await async_print("\n输入被中断。程序即将退出。")
            logger.info("[Main] 用户中断了 API Key 输入。")
            return # 退出 main 函数。
        # 再次检查用户是否确实输入了 Key。
        if not api_key:
            await async_print("\n错误：未提供 API Key。程序无法启动，即将退出。")
            logger.critical("[Main] 用户未提供 API Key。")
            return
        logger.info("[Main] 已通过手动输入获取 API Key。")
        await async_print("API Key 已获取。")

    # --- 初始化 Agent ---
    agent = None
    try:
        # 实例化 Agent 核心类。
        # 在这里可以集中配置 Agent 的关键参数，如 LLM 模型名称、规划重试次数、记忆容量等。
        agent = CircuitDesignAgent(
            api_key=api_key,
            model_name="glm-4-flash", # 可以根据需要更换为其他兼容的智谱模型。
            planning_llm_retries=1,   # 配置规划阶段失败时的重试次数。
            max_short_term_items=25   # 配置短期记忆的最大条目数。
        )
        await async_print("\n🎉 Agent 初始化成功！已准备就绪。")
        # 提供一些示例指令，帮助用户开始使用。
        await async_print("\n您可以尝试以下指令:")
        await async_print("  - '给我加个1k电阻R1K和3V电池B3V'")
        await async_print("  - '连接R1K和B3V'")
        await async_print("  - '电路现在什么样？'")
        await async_print("  - '尝试连接 R1K 和一个不存在的元件 XYZ'") # 用于演示工具失败和提前中止。
        await async_print("  - '清空电路'")
        await async_print("  - '你好，今天天气怎么样？'") # 用于演示不需要工具的直接回复。
        await async_print("  - 输入 '退出' 来结束程序")
        await async_print("-" * 70)
    except Exception as e:
        # 如果 Agent 初始化过程中发生任何错误，这是致命的，记录错误并退出程序。
        logger.critical(f"[Main] Agent 初始化失败: {e}", exc_info=True)
        await async_print(f"\n🔴 Agent 初始化失败！错误: {e}。请检查日志和配置。程序即将退出。")
        return # 从 main 函数返回，程序将结束。

    # --- 主交互循环 ---
    # 这个循环负责持续接收用户输入，并调用 Agent 的核心处理方法来响应。
    try:
        while True:
            try:
                user_input = ""
                try:
                    # 获取用户输入。再次注意，`input()` 是同步阻塞的。
                    user_input = input("用户 > ").strip()
                except (EOFError, KeyboardInterrupt):
                    # 处理用户在输入提示符处中断的情况 (Ctrl+D 或 Ctrl+C)。
                    raise # 重新抛出，由外层的 try...except 捕获并处理退出逻辑。

                # 检查用户是否输入了退出指令。
                if user_input.lower() in ['退出', 'quit', 'exit', '再见', '结束', 'bye']:
                    await async_print("\n收到退出指令。感谢您的使用！👋")
                    logger.info("[Main] 收到退出指令，结束交互循环。")
                    break # 跳出 while 循环，结束程序。

                # 如果用户只输入了空白符，则忽略并等待下一条指令。
                if not user_input:
                    continue

                # **调用 Agent 的核心处理方法 `process_user_request`**
                # 这是一个异步调用，会执行完整的 Agentic 循环。
                start_process_time = time.monotonic() # 记录单次请求处理的开始时间。
                response = await agent.process_user_request(user_input)
                process_duration = time.monotonic() - start_process_time # 计算处理耗时。

                # 打印 Agent 返回的完整响应字符串 (通常包含 `<think>` 和正式回复)。
                await async_print(f"\n📝 Agent 回复 (耗时: {process_duration:.3f} 秒):")
                await async_print(response)
                await async_print("-" * 70) # 使用分隔符使交互界面更清晰。

            except KeyboardInterrupt:
                # 如果在 Agent 处理请求的过程中 (即在 `await agent.process_user_request` 执行期间) 用户按下 Ctrl+C。
                await async_print("\n用户操作被中断。")
                logger.info("[Main] 用户中断了当前请求的处理。")
                # 在这种情况下，选择退出整个交互循环。
                break
            except EOFError:
                # 如果输入流意外结束 (例如，当从文件或管道读取输入时)。
                await async_print("\n输入流意外结束。")
                logger.info("[Main] 输入流结束 (EOF)。")
                break
            except Exception as loop_err:
                # 捕获在处理单个用户请求过程中，Agent 内部未能捕获的意外错误。
                # 这个捕获块确保单个请求的失败不会导致整个应用程序崩溃。
                logger.error(f"[Main] 处理指令 '{user_input[:50]}...' 时发生意外错误: {loop_err}", exc_info=True)
                await async_print(f"\n🔴 Agent 运行时错误:")
                # 生成一个安全的错误报告给用户，避免再次调用可能出错的 Agent 逻辑。
                error_report = f"<think>处理指令 '{user_input[:50]}...' 时发生内部错误: {loop_err}\n{traceback.format_exc()}</think>\n\n抱歉，我在执行您的指令时遇到了意外问题 ({loop_err})！我已经记录了详细的技术信息。请尝试其他指令或检查日志。"
                await async_print(error_report)
                await async_print("-" * 70)
                # 发生错误后，继续下一次循环，尝试处理用户的下一条指令。
                continue

    except Exception as outer_loop_err:
        # 这个捕获块处理在主交互循环本身之外发生的、更严重的未处理异常。
        logger.critical(f"[Main] 主交互循环外发生未处理异常: {outer_loop_err}", exc_info=True)
        await async_print(f"\n🔴 严重系统错误导致交互循环终止: {outer_loop_err}。")
    finally:
        # 无论循环是如何结束的（正常退出或异常退出），都执行这里的清理代码。
        logger.info("[Main] 主交互循环结束。")
        await async_print("\n正在关闭 Agent...")
        # 如果 Agent 类定义了任何清理方法 (例如关闭网络连接、保存状态等)，
        # 可以在这里调用，例如: `await agent.cleanup()`。

# --- 用于 Jupyter/IPython 环境的辅助函数 ---
# 在 Jupyter Notebook 或类似的交互式环境中，直接运行 `asyncio.run(main())` 可能会与环境已有的事件循环冲突。
# 这个辅助函数提供了一种在这些环境中更安全地启动 Agent 交互的方式。
async def run_agent_in_jupyter():
    """
    在 Jupyter/IPython 环境中安全启动 Agent 交互循环的辅助函数。
    使用方法：在 Notebook 的一个 cell 中执行 `await run_agent_in_jupyter()`。
    """
    print("正在尝试以 Jupyter/IPython 兼容模式启动 Agent...")
    print("请在下方的输入提示处输入指令。输入 '退出' 结束。")
    # 直接调用并等待 `main` 协程完成。
    try:
        await main()
    except Exception as e:
        # 捕获在 Jupyter 模式下运行时可能发生的顶层异常。
        print(f"\n🔴 Agent 在 Jupyter 模式下运行时遇到错误: {e}")
        logger.error(f"在 Jupyter 模式下运行 Agent 时出错: {e}", exc_info=True)
    finally:
        print("Agent 交互已结束 (Jupyter 模式)。")


# --- 标准 Python 脚本入口点 ---
# if __name__ == "__main__":
#     # 首先尝试检测当前是否运行在 IPython 或 Jupyter 环境中。
#     # 这是因为这些环境可能已经有正在运行的 asyncio 事件循环，
#     # 直接调用 `asyncio.run()` 可能会导致冲突。
#     is_jupyter_env = False
#     try:
#         # `get_ipython` 是 IPython 环境提供的一个内置函数。
#         shell = get_ipython().__class__.__name__
#         # 'ZMQInteractiveShell' 通常表示在 Jupyter Notebook/Lab 或 QtConsole 中运行。
#         # 'TerminalInteractiveShell' 表示在终端中通过 `ipython` 命令启动。
#         if shell == 'ZMQInteractiveShell':
#             is_jupyter_env = True
#             print("检测到 Jupyter/IPython (ZMQ) 环境。")
#             print("请在 Notebook cell 中执行 `await run_agent_in_jupyter()` 来启动 Agent 交互。")
#             # 在 Jupyter Notebook 环境中，不自动启动 `main`，而是提示用户手动调用辅助函数。
#         elif shell == 'TerminalInteractiveShell':
#             # 在终端 IPython 中，行为与标准脚本类似，可以尝试直接运行。
#             print("检测到终端 IPython 环境。将尝试作为标准脚本启动...")
#             is_jupyter_env = False # 标记为非特殊 Jupyter 环境
#         else:
#             # 其他类型的 IPython shell 比较少见，也按标准脚本处理。
#             print(f"检测到非典型的 IPython shell ({shell})。将尝试作为标准脚本启动...")
#             is_jupyter_env = False
#     except NameError:
#         # 如果 `get_ipython` 未定义，说明是在标准的 Python 解释器中运行。
#         is_jupyter_env = False

#     # 如果不是特殊的 Jupyter ZMQ 环境，则按标准脚本方式启动。
#     if not is_jupyter_env:
#         print("正在以标准 Python 脚本模式启动 Agent...")
#         try:
#             # 使用 `asyncio.run()` 来启动并运行顶层的异步 `main` 函数。
#             # 这是在标准 Python 文件中运行异步代码的推荐方式，它会负责事件循环的创建和管理。
#             asyncio.run(main())
#         except KeyboardInterrupt:
#             # 捕获在 `asyncio.run()` 执行期间用户按下的 Ctrl+C。
#             print("\n程序被用户强制退出 (KeyboardInterrupt)。")
#             logger.info("[Main Script] 程序被 KeyboardInterrupt 中断。")
#         except Exception as e:
#             # 捕获在 `asyncio.run()` 执行期间可能发生的其他未被处理的顶层异常。
#             print(f"\n程序因顶层错误而意外退出: {e}")
#             logger.critical(f"脚本执行期间发生顶层异常: {e}", exc_info=True)
#         finally:
#             # 无论程序如何退出，都打印结束信息。
#             print("Agent 程序已关闭。")


In [ ]:
await main()
